In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW13OT-MakeConcordance-1Chronicles.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

FirstChronicles01 = [
    ['1Chronicles','1Chr','01','001','1',"Adam, Sheth, Enosh,"],
['1Chronicles','1Chr','01','002','2',"Kenan, Mahalaleel, Jered,"],
['1Chronicles','1Chr','01','003','3',"Henoch, Methuselah, Lamech,"],
['1Chronicles','1Chr','01','004','4',"Noah, Shem, Ham, and Japheth."],
['1Chronicles','1Chr','01','005','5',"The sons of Japheth; Gomer, and Magog, and Madai, and Javan, and Tubal, and Meshech, and Tiras."],
['1Chronicles','1Chr','01','006','6',"And the sons of Gomer; Ashchenaz, and Riphath, and Togarmah."],
['1Chronicles','1Chr','01','007','7',"And the sons of Javan; Elishah, and Tarshish, Kittim, and Dodanim."],
['1Chronicles','1Chr','01','008','8',"The sons of Ham; Cush, and Mizraim, Put, and Canaan."],
['1Chronicles','1Chr','01','009','9',"And the sons of Cush; Seba, and Havilah, and Sabta, and Raamah, and Sabtecha. And the sons of Raamah; Sheba, and Dedan."],
['1Chronicles','1Chr','01','010','10',"And Cush begat Nimrod: he began to be mighty upon the earth."],
['1Chronicles','1Chr','01','011','11',"And Mizraim begat Ludim, and Anamim, and Lehabim, and Naphtuhim,"],
['1Chronicles','1Chr','01','012','12',"And Pathrusim, and Casluhim, (of whom came the Philistines,) and Caphthorim."],
['1Chronicles','1Chr','01','013','13',"And Canaan begat Zidon his firstborn, and Heth,"],
['1Chronicles','1Chr','01','014','14',"The Jebusite also, and the Amorite, and the Girgashite,"],
['1Chronicles','1Chr','01','015','15',"And the Hivite, and the Arkite, and the Sinite,"],
['1Chronicles','1Chr','01','016','16',"And the Arvadite, and the Zemarite, and the Hamathite."],
['1Chronicles','1Chr','01','017','17',"The sons of Shem; Elam, and Asshur, and Arphaxad, and Lud, and Aram, and Uz, and Hul, and Gether, and Meshech."],
['1Chronicles','1Chr','01','018','18',"And Arphaxad begat Shelah, and Shelah begat Eber."],
['1Chronicles','1Chr','01','019','19',"And unto Eber were born two sons: the name of the one \emph{was} Peleg; because in his days the earth was divided: and his brother's name \emph{was} Joktan."],
['1Chronicles','1Chr','01','020','20',"And Joktan begat Almodad, and Sheleph, and Hazarmaveth, and Jerah,"],
['1Chronicles','1Chr','01','021','21',"Hadoram also, and Uzal, and Diklah,"],
['1Chronicles','1Chr','01','022','22',"And Ebal, and Abimael, and Sheba,"],
['1Chronicles','1Chr','01','023','23',"And Ophir, and Havilah, and Jobab. All these \emph{were} the sons of Joktan."],
['1Chronicles','1Chr','01','024','24',"Shem, Arphaxad, Shelah,"],
['1Chronicles','1Chr','01','025','25',"Eber, Peleg, Reu,"],
['1Chronicles','1Chr','01','026','26',"Serug, Nahor, Terah,"],
['1Chronicles','1Chr','01','027','27',"Abram; the same \emph{is} Abraham."],
['1Chronicles','1Chr','01','028','28',"The sons of Abraham; Isaac, and Ishmael."],
['1Chronicles','1Chr','01','029','29',"These \emph{are} their generations: The firstborn of Ishmael, Nebaioth; then Kedar, and Adbeel, and Mibsam,"],
['1Chronicles','1Chr','01','030','30',"Mishma, and Dumah, Massa, Hadad, and Tema,"],
['1Chronicles','1Chr','01','031','31',"Jetur, Naphish, and Kedemah. These are the sons of Ishmael."],
['1Chronicles','1Chr','01','032','32',"Now the sons of Keturah, Abraham's concubine: she bare Zimran, and Jokshan, and Medan, and Midian, and Ishbak, and Shuah. And the sons of Jokshan; Sheba, and Dedan."],
['1Chronicles','1Chr','01','033','33',"And the sons of Midian; Ephah, and Epher, and Henoch, and Abida, and Eldaah. All these \emph{are} the sons of Keturah."],
['1Chronicles','1Chr','01','034','34',"And Abraham begat Isaac. The sons of Isaac; Esau and Israel."],
['1Chronicles','1Chr','01','035','35',"The sons of Esau; Eliphaz, Reuel, and Jeush, and Jaalam, and Korah."],
['1Chronicles','1Chr','01','036','36',"The sons of Eliphaz; Teman, and Omar, Zephi, and Gatam, Kenaz, and Timna, and Amalek."],
['1Chronicles','1Chr','01','037','37',"The sons of Reuel; Nahath, Zerah, Shammah, and Mizzah."],
['1Chronicles','1Chr','01','038','38',"And the sons of Seir; Lotan, and Shobal, and Zibeon, and Anah, and Dishon, and Ezer, and Dishan."],
['1Chronicles','1Chr','01','039','39',"And the sons of Lotan; Hori, and Homam: and Timna \emph{was} Lotan's sister."],
['1Chronicles','1Chr','01','040','40',"The sons of Shobal; Alian, and Manahath, and Ebal, Shephi, and Onam. And the sons of Zibeon; Aiah, and Anah."],
['1Chronicles','1Chr','01','041','41',"The sons of Anah; Dishon. And the sons of Dishon; Amram, and Eshban, and Ithran, and Cheran."],
['1Chronicles','1Chr','01','042','42',"The sons of Ezer; Bilhan, and Zavan, \emph{and} Jakan. The sons of Dishan; Uz, and Aran."],
['1Chronicles','1Chr','01','043','43',"Now these \emph{are} the kings that reigned in the land of Edom before \emph{any} king reigned over the children of Israel; Bela the son of Beor: and the name of his city \emph{was} Dinhabah."],
['1Chronicles','1Chr','01','044','44',"And when Bela was dead, Jobab the son of Zerah of Bozrah reigned in his stead."],
['1Chronicles','1Chr','01','045','45',"And when Jobab was dead, Husham of the land of the Temanites reigned in his stead."],
['1Chronicles','1Chr','01','046','46',"And when Husham was dead, Hadad the son of Bedad, which smote Midian in the field of Moab, reigned in his stead: and the name of his city \emph{was} Avith."],
['1Chronicles','1Chr','01','047','47',"And when Hadad was dead, Samlah of Masrekah reigned in his stead."],
['1Chronicles','1Chr','01','048','48',"And when Samlah was dead, Shaul of Rehoboth by the river reigned in his stead."],
['1Chronicles','1Chr','01','049','49',"And when Shaul was dead, Baal-hanan the son of Achbor reigned in his stead."],
['1Chronicles','1Chr','01','050','50',"And when Baal-hanan was dead, Hadad reigned in his stead: and the name of his city \emph{was} Pai; and his wife's name \emph{was} Mehetabel, the daughter of Matred, the daughter of Mezahab."],
['1Chronicles','1Chr','01','051','51',"Hadad died also. And the dukes of Edom were; duke Timnah, duke Aliah, duke Jetheth,"],
['1Chronicles','1Chr','01','052','52',"Duke Aholibamah, duke Elah, duke Pinon,"],
['1Chronicles','1Chr','01','053','53',"Duke Kenaz, duke Teman, duke Mibzar,"],
['1Chronicles','1Chr','01','054','54',"Duke Magdiel, duke Iram. These \emph{are} the dukes of Edom."]
]

FirstChronicles02 = [
    ['1Chronicles','1Chr','02','001','1',"These \emph{are} the sons of Israel; Reuben, Simeon, Levi, and Judah, Issachar, and Zebulun,"],
['1Chronicles','1Chr','02','002','2',"Dan, Joseph, and Benjamin, Naphtali, Gad, and Asher."],
['1Chronicles','1Chr','02','003','3',"The sons of Judah; Er, and Onan, and Shelah: \emph{which} three were born unto him of the daughter of Shua the Canaanitess. And Er, the firstborn of Judah, was evil in the sight of the LORD; and he slew him."],
['1Chronicles','1Chr','02','004','4',"And Tamar his daughter in law bare him Pharez and Zerah. All the sons of Judah \emph{were} five."],
['1Chronicles','1Chr','02','005','5',"The sons of Pharez; Hezron, and Hamul."],
['1Chronicles','1Chr','02','006','6',"And the sons of Zerah; Zimri, and Ethan, and Heman, and Calcol, and Dara: five of them in all."],
['1Chronicles','1Chr','02','007','7',"And the sons of Carmi; Achar, the troubler of Israel, who transgressed in the thing accursed."],
['1Chronicles','1Chr','02','008','8',"And the sons of Ethan; Azariah."],
['1Chronicles','1Chr','02','009','9',"The sons also of Hezron, that were born unto him; Jerahmeel, and Ram, and Chelubai."],
['1Chronicles','1Chr','02','010','10',"And Ram begat Amminadab; and Amminadab begat Nahshon, prince of the children of Judah;"],
['1Chronicles','1Chr','02','011','11',"And Nahshon begat Salma, and Salma begat Boaz,"],
['1Chronicles','1Chr','02','012','12',"And Boaz begat Obed, and Obed begat Jesse,"],
['1Chronicles','1Chr','02','013','13',"And Jesse begat his firstborn Eliab, and Abinadab the second, and Shimma the third,"],
['1Chronicles','1Chr','02','014','14',"Nethaneel the fourth, Raddai the fifth,"],
['1Chronicles','1Chr','02','015','15',"Ozem the sixth, David the seventh:"],
['1Chronicles','1Chr','02','016','16',"Whose sisters \emph{were} Zeruiah, and Abigail. And the sons of Zeruiah; Abishai, and Joab, and Asahel, three."],
['1Chronicles','1Chr','02','017','17',"And Abigail bare Amasa: and the father of Amasa \emph{was} Jether the Ishmeelite."],
['1Chronicles','1Chr','02','018','18',"And Caleb the son of Hezron begat \emph{children} of Azubah \emph{his} wife, and of Jerioth: her sons \emph{are} these; Jesher, and Shobab, and Ardon."],
['1Chronicles','1Chr','02','019','19',"And when Azubah was dead, Caleb took unto him Ephrath, which bare him Hur."],
['1Chronicles','1Chr','02','020','20',"And Hur begat Uri, and Uri begat Bezaleel."],
['1Chronicles','1Chr','02','021','21',"And afterward Hezron went in to the daughter of Machir the father of Gilead, whom he married when he \emph{was} threescore years old; and she bare him Segub."],
['1Chronicles','1Chr','02','022','22',"And Segub begat Jair, who had three and twenty cities in the land of Gilead."],
['1Chronicles','1Chr','02','023','23',"And he took Geshur, and Aram, with the towns of Jair, from them, with Kenath, and the towns thereof, \emph{even} threescore cities. All these \emph{belonged} \emph{to} the sons of Machir the father of Gilead."],
['1Chronicles','1Chr','02','024','24',"And after that Hezron was dead in Caleb-ephratah, then Abiah Hezron's wife bare him Ashur the father of Tekoa."],
['1Chronicles','1Chr','02','025','25',"And the sons of Jerahmeel the firstborn of Hezron were, Ram the firstborn, and Bunah, and Oren, and Ozem, \emph{and} Ahijah."],
['1Chronicles','1Chr','02','026','26',"Jerahmeel had also another wife, whose name \emph{was} Atarah; she \emph{was} the mother of Onam."],
['1Chronicles','1Chr','02','027','27',"And the sons of Ram the firstborn of Jerahmeel were, Maaz, and Jamin, and Eker."],
['1Chronicles','1Chr','02','028','28',"And the sons of Onam were, Shammai, and Jada. And the sons of Shammai; Nadab, and Abishur."],
['1Chronicles','1Chr','02','029','29',"And the name of the wife of Abishur \emph{was} Abihail, and she bare him Ahban, and Molid."],
['1Chronicles','1Chr','02','030','30',"And the sons of Nadab; Seled, and Appaim: but Seled died without children."],
['1Chronicles','1Chr','02','031','31',"And the sons of Appaim; Ishi. And the sons of Ishi; Sheshan. And the children of Sheshan; Ahlai."],
['1Chronicles','1Chr','02','032','32',"And the sons of Jada the brother of Shammai; Jether, and Jonathan: and Jether died without children."],
['1Chronicles','1Chr','02','033','33',"And the sons of Jonathan; Peleth, and Zaza. These were the sons of Jerahmeel."],
['1Chronicles','1Chr','02','034','34',"Now Sheshan had no sons, but daughters. And Sheshan had a servant, an Egyptian, whose name \emph{was} Jarha."],
['1Chronicles','1Chr','02','035','35',"And Sheshan gave his daughter to Jarha his servant to wife; and she bare him Attai."],
['1Chronicles','1Chr','02','036','36',"And Attai begat Nathan, and Nathan begat Zabad,"],
['1Chronicles','1Chr','02','037','37',"And Zabad begat Ephlal, and Ephlal begat Obed,"],
['1Chronicles','1Chr','02','038','38',"And Obed begat Jehu, and Jehu begat Azariah,"],
['1Chronicles','1Chr','02','039','39',"And Azariah begat Helez, and Helez begat Eleasah,"],
['1Chronicles','1Chr','02','040','40',"And Eleasah begat Sisamai, and Sisamai begat Shallum,"],
['1Chronicles','1Chr','02','041','41',"And Shallum begat Jekamiah, and Jekamiah begat Elishama."],
['1Chronicles','1Chr','02','042','42',"Now the sons of Caleb the brother of Jerahmeel \emph{were}, Mesha his firstborn, which was the father of Ziph; and the sons of Mareshah the father of Hebron."],
['1Chronicles','1Chr','02','043','43',"And the sons of Hebron; Korah, and Tappuah, and Rekem, and Shema."],
['1Chronicles','1Chr','02','044','44',"And Shema begat Raham, the father of Jorkoam: and Rekem begat Shammai."],
['1Chronicles','1Chr','02','045','45',"And the son of Shammai \emph{was} Maon: and Maon \emph{was} the father of Beth-zur."],
['1Chronicles','1Chr','02','046','46',"And Ephah, Caleb's concubine, bare Haran, and Moza, and Gazez: and Haran begat Gazez."],
['1Chronicles','1Chr','02','047','47',"And the sons of Jahdai; Regem, and Jotham, and Gesham, and Pelet, and Ephah, and Shaaph."],
['1Chronicles','1Chr','02','048','48',"Maachah, Caleb's concubine, bare Sheber, and Tirhanah."],
['1Chronicles','1Chr','02','049','49',"She bare also Shaaph the father of Madmannah, Sheva the father of Machbenah, and the father of Gibea: and the daughter of Caleb \emph{was} Achsa."],
['1Chronicles','1Chr','02','050','50',"These were the sons of Caleb the son of Hur, the firstborn of Ephratah; Shobal the father of Kirjath-jearim,"],
['1Chronicles','1Chr','02','051','51',"Salma the father of Beth-lehem, Hareph the father of Beth-gader."],
['1Chronicles','1Chr','02','052','52',"And Shobal the father of Kirjath-jearim had sons; Haroeh, \emph{and} half of the Manahethites."],
['1Chronicles','1Chr','02','053','53',"And the families of Kirjath-jearim; the Ithrites, and the Puhites, and the Shumathites, and the Mishraites; of them came the Zareathites, and the Eshtaulites."],
['1Chronicles','1Chr','02','054','54',"The sons of Salma; Beth-lehem, and the Netophathites, Ataroth, the house of Joab, and half of the Manahethites, the Zorites."],
['1Chronicles','1Chr','02','055','55',"And the families of the scribes which dwelt at Jabez; the Tirathites, the Shimeathites, \emph{and} Suchathites. These \emph{are} the Kenites that came of Hemath, the father of the house of Rechab."]
]

FirstChronicles03 = [
['1Chronicles','1Chr','03','001','1',"Now these were the sons of David, which were born unto him in Hebron; the firstborn Amnon, of Ahinoam the Jezreelitess; the second Daniel, of Abigail the Carmelitess:"],
['1Chronicles','1Chr','03','002','2',"The third, Absalom the son of Maachah the daughter of Talmai king of Geshur: the fourth, Adonijah the son of Haggith:"],
['1Chronicles','1Chr','03','003','3',"The fifth, Shephatiah of Abital: the sixth, Ithream by Eglah his wife."],
['1Chronicles','1Chr','03','004','4',"\emph{These} six were born unto him in Hebron; and there he reigned seven years and six months: and in Jerusalem he reigned thirty and three years."],
['1Chronicles','1Chr','03','005','5',"And these were born unto him in Jerusalem; Shimea, and Shobab, and Nathan, and Solomon, four, of Bath-shua the daughter of Ammiel:"],
['1Chronicles','1Chr','03','006','6',"Ibhar also, and Elishama, and Eliphelet,"],
['1Chronicles','1Chr','03','007','7',"And Nogah, and Nepheg, and Japhia,"],
['1Chronicles','1Chr','03','008','8',"And Elishama, and Eliada, and Eliphelet, nine."],
['1Chronicles','1Chr','03','009','9',"\emph{These} \emph{were} all the sons of David, beside the sons of the concubines, and Tamar their sister."],
['1Chronicles','1Chr','03','010','10',"And Solomon's son \emph{was} Rehoboam, Abia his son, Asa his son, Jehoshaphat his son,"],
['1Chronicles','1Chr','03','011','11',"Joram his son, Ahaziah his son, Joash his son,"],
['1Chronicles','1Chr','03','012','12',"Amaziah his son, Azariah his son, Jotham his son,"],
['1Chronicles','1Chr','03','013','13',"Ahaz his son, Hezekiah his son, Manasseh his son,"],
['1Chronicles','1Chr','03','014','14',"Amon his son, Josiah his son."],
['1Chronicles','1Chr','03','015','15',"And the sons of Josiah \emph{were}, the firstborn Johanan, the second Jehoiakim, the third Zedekiah, the fourth Shallum."],
['1Chronicles','1Chr','03','016','16',"And the sons of Jehoiakim: Jeconiah his son, Zedekiah his son."],
['1Chronicles','1Chr','03','017','17',"And the sons of Jeconiah; Assir, Salathiel his son,"],
['1Chronicles','1Chr','03','018','18',"Malchiram also, and Pedaiah, and Shenazar, Jecamiah, Hoshama, and Nedabiah."],
['1Chronicles','1Chr','03','019','19',"And the sons of Pedaiah \emph{were}, Zerubbabel, and Shimei: and the sons of Zerubbabel; Meshullam, and Hananiah, and Shelomith their sister:"],
['1Chronicles','1Chr','03','020','20',"And Hashubah, and Ohel, and Berechiah, and Hasadiah, Jushab-hesed, five."],
['1Chronicles','1Chr','03','021','21',"And the sons of Hananiah; Pelatiah, and Jesaiah: the sons of Rephaiah, the sons of Arnan, the sons of Obadiah, the sons of Shechaniah."],
['1Chronicles','1Chr','03','022','22',"And the sons of Shechaniah; Shemaiah: and the sons of Shemaiah; Hattush, and Igeal, and Bariah, and Neariah, and Shaphat, six."],
['1Chronicles','1Chr','03','023','23',"And the sons of Neariah; Elioenai, and Hezekiah, and Azrikam, three."],
['1Chronicles','1Chr','03','024','24',"And the sons of Elioenai \emph{were}, Hodaiah, and Eliashib, and Pelaiah, and Akkub, and Johanan, and Dalaiah, and Anani, seven."]
]

FirstChronicles04 = [
    ['1Chronicles','1Chr','04','001','1',"The sons of Judah; Pharez, Hezron, and Carmi, and Hur, and Shobal."],
['1Chronicles','1Chr','04','002','2',"And Reaiah the son of Shobal begat Jahath; and Jahath begat Ahumai, and Lahad. These \emph{are} the families of the Zorathites."],
['1Chronicles','1Chr','04','003','3',"And these \emph{were} \emph{of} the father of Etam; Jezreel, and Ishma, and Idbash: and the name of their sister \emph{was} Hazelelponi:"],
['1Chronicles','1Chr','04','004','4',"And Penuel the father of Gedor, and Ezer the father of Hushah. These \emph{are} the sons of Hur, the firstborn of Ephratah, the father of Beth-lehem."],
['1Chronicles','1Chr','04','005','5',"And Ashur the father of Tekoa had two wives, Helah and Naarah."],
['1Chronicles','1Chr','04','006','6',"And Naarah bare him Ahuzam, and Hepher, and Temeni, and Haahashtari. These \emph{were} the sons of Naarah."],
['1Chronicles','1Chr','04','007','7',"And the sons of Helah \emph{were}, Zereth, and Jezoar, and Ethnan."],
['1Chronicles','1Chr','04','008','8',"And Coz begat Anub, and Zobebah, and the families of Aharhel the son of Harum."],
['1Chronicles','1Chr','04','009','9',"And Jabez was more honourable than his brethren: and his mother called his name Jabez, saying, Because I bare him with sorrow."],
['1Chronicles','1Chr','04','010','10',"And Jabez called on the God of Israel, saying, Oh that thou wouldest bless me indeed, and enlarge my coast, and that thine hand might be with me, and that thou wouldest keep \emph{me} from evil, that it may not grieve me! And God granted him that which he requested."],
['1Chronicles','1Chr','04','011','11',"And Chelub the brother of Shuah begat Mehir, which \emph{was} the father of Eshton."],
['1Chronicles','1Chr','04','012','12',"And Eshton begat Beth-rapha, and Paseah, and Tehinnah the father of Ir-nahash. These \emph{are} the men of Rechah."],
['1Chronicles','1Chr','04','013','13',"And the sons of Kenaz; Othniel, and Seraiah: and the sons of Othniel; Hathath."],
['1Chronicles','1Chr','04','014','14',"And Meonothai begat Ophrah: and Seraiah begat Joab, the father of the valley of Charashim; for they were craftsmen."],
['1Chronicles','1Chr','04','015','15',"And the sons of Caleb the son of Jephunneh; Iru, Elah, and Naam: and the sons of Elah, even Kenaz."],
['1Chronicles','1Chr','04','016','16',"And the sons of Jehaleleel; Ziph, and Ziphah, Tiria, and Asareel."],
['1Chronicles','1Chr','04','017','17',"And the sons of Ezra \emph{were}, Jether, and Mered, and Epher, and Jalon: and she bare Miriam, and Shammai, and Ishbah the father of Eshtemoa."],
['1Chronicles','1Chr','04','018','18',"And his wife Jehudijah bare Jered the father of Gedor, and Heber the father of Socho, and Jekuthiel the father of Zanoah. And these \emph{are} the sons of Bithiah the daughter of Pharaoh, which Mered took."],
['1Chronicles','1Chr','04','019','19',"And the sons of \emph{his} wife Hodiah the sister of Naham, the father of Keilah the Garmite, and Eshtemoa the Maachathite."],
['1Chronicles','1Chr','04','020','20',"And the sons of Shimon \emph{were}, Amnon, and Rinnah, Ben-hanan, and Tilon. And the sons of Ishi \emph{were}, Zoheth, and Ben-zoheth."],
['1Chronicles','1Chr','04','021','21',"The sons of Shelah the son of Judah \emph{were}, Er the father of Lecah, and Laadah the father of Mareshah, and the families of the house of them that wrought fine linen, of the house of Ashbea,"],
['1Chronicles','1Chr','04','022','22',"And Jokim, and the men of Chozeba, and Joash, and Saraph, who had the dominion in Moab, and Jashubi-lehem. And \emph{these} \emph{are} ancient things."],
['1Chronicles','1Chr','04','023','23',"These \emph{were} the potters, and those that dwelt among plants and hedges: there they dwelt with the king for his work."],
['1Chronicles','1Chr','04','024','24',"The sons of Simeon \emph{were}, Nemuel, and Jamin, Jarib, Zerah, \emph{and} Shaul:"],
['1Chronicles','1Chr','04','025','25',"Shallum his son, Mibsam his son, Mishma his son."],
['1Chronicles','1Chr','04','026','26',"And the sons of Mishma; Hamuel his son, Zacchur his son, Shimei his son."],
['1Chronicles','1Chr','04','027','27',"And Shimei had sixteen sons and six daughters; but his brethren had not many children, neither did all their family multiply, like to the children of Judah."],
['1Chronicles','1Chr','04','028','28',"And they dwelt at Beer-sheba, and Moladah, and Hazar-shual,"],
['1Chronicles','1Chr','04','029','29',"And at Bilhah, and at Ezem, and at Tolad,"],
['1Chronicles','1Chr','04','030','30',"And at Bethuel, and at Hormah, and at Ziklag,"],
['1Chronicles','1Chr','04','031','31',"And at Beth-marcaboth, and Hazar-susim, and at Beth-birei, and at Shaaraim. These \emph{were} their cities unto the reign of David."],
['1Chronicles','1Chr','04','032','32',"And their villages \emph{were}, Etam, and Ain, Rimmon, and Tochen, and Ashan, five cities:"],
['1Chronicles','1Chr','04','033','33',"And all their villages that \emph{were} round about the same cities, unto Baal. These \emph{were} their habitations, and their genealogy."],
['1Chronicles','1Chr','04','034','34',"And Meshobab, and Jamlech, and Joshah the son of Amaziah,"],
['1Chronicles','1Chr','04','035','35',"And Joel, and Jehu the son of Josibiah, the son of Seraiah, the son of Asiel,"],
['1Chronicles','1Chr','04','036','36',"And Elioenai, and Jaakobah, and Jeshohaiah, and Asaiah, and Adiel, and Jesimiel, and Benaiah,"],
['1Chronicles','1Chr','04','037','37',"And Ziza the son of Shiphi, the son of Allon, the son of Jedaiah, the son of Shimri, the son of Shemaiah;"],
['1Chronicles','1Chr','04','038','38',"These mentioned by \emph{their} names \emph{were} princes in their families: and the house of their fathers increased greatly."],
['1Chronicles','1Chr','04','039','39',"And they went to the entrance of Gedor, \emph{even} unto the east side of the valley, to seek pasture for their flocks."],
['1Chronicles','1Chr','04','040','40',"And they found fat pasture and good, and the land \emph{was} wide, and quiet, and peaceable; for \emph{they} of Ham had dwelt there of old."],
['1Chronicles','1Chr','04','041','41',"And these written by name came in the days of Hezekiah king of Judah, and smote their tents, and the habitations that were found there, and destroyed them utterly unto this day, and dwelt in their rooms: because \emph{there} \emph{was} pasture there for their flocks."],
['1Chronicles','1Chr','04','042','42',"And \emph{some} of them, \emph{even} of the sons of Simeon, five hundred men, went to mount Seir, having for their captains Pelatiah, and Neariah, and Rephaiah, and Uzziel, the sons of Ishi."],
['1Chronicles','1Chr','04','043','43',"And they smote the rest of the Amalekites that were escaped, and dwelt there unto this day."]
]

FirstChronicles05 = [
    ['1Chronicles','1Chr','05','001','1',"Now the sons of Reuben the firstborn of Israel, (for he \emph{was} the firstborn; but, forasmuch as he defiled his father's bed, his birthright was given unto the sons of Joseph the son of Israel: and the genealogy is not to be reckoned after the birthright."],
['1Chronicles','1Chr','05','002','2',"For Judah prevailed above his brethren, and of him \emph{came} the chief ruler; but the birthright \emph{was} Joseph's:)"],
['1Chronicles','1Chr','05','003','3',"The sons, \emph{I} \emph{say}, of Reuben the firstborn of Israel \emph{were}, Hanoch, and Pallu, Hezron, and Carmi."],
['1Chronicles','1Chr','05','004','4',"The sons of Joel; Shemaiah his son, Gog his son, Shimei his son,"],
['1Chronicles','1Chr','05','005','5',"Micah his son, Reaia his son, Baal his son,"],
['1Chronicles','1Chr','05','006','6',"Beerah his son, whom Tilgath-pilneser king of Assyria carried away \emph{captive}: he \emph{was} prince of the Reubenites."],
['1Chronicles','1Chr','05','007','7',"And his brethren by their families, when the genealogy of their generations was reckoned, \emph{were} the chief, Jeiel, and Zechariah,"],
['1Chronicles','1Chr','05','008','8',"And Bela the son of Azaz, the son of Shema, the son of Joel, who dwelt in Aroer, even unto Nebo and Baal-meon:"],
['1Chronicles','1Chr','05','009','9',"And eastward he inhabited unto the entering in of the wilderness from the river Euphrates: because their cattle were multiplied in the land of Gilead."],
['1Chronicles','1Chr','05','010','10',"And in the days of Saul they made war with the Hagarites, who fell by their hand: and they dwelt in their tents throughout all the east \emph{land} of Gilead."],
['1Chronicles','1Chr','05','011','11',"And the children of Gad dwelt over against them, in the land of Bashan unto Salchah:"],
['1Chronicles','1Chr','05','012','12',"Joel the chief, and Shapham the next, and Jaanai, and Shaphat in Bashan."],
['1Chronicles','1Chr','05','013','13',"And their brethren of the house of their fathers \emph{were}, Michael, and Meshullam, and Sheba, and Jorai, and Jachan, and Zia, and Heber, seven."],
['1Chronicles','1Chr','05','014','14',"These \emph{are} the children of Abihail the son of Huri, the son of Jaroah, the son of Gilead, the son of Michael, the son of Jeshishai, the son of Jahdo, the son of Buz;"],
['1Chronicles','1Chr','05','015','15',"Ahi the son of Abdiel, the son of Guni, chief of the house of their fathers."],
['1Chronicles','1Chr','05','016','16',"And they dwelt in Gilead in Bashan, and in her towns, and in all the suburbs of Sharon, upon their borders."],
['1Chronicles','1Chr','05','017','17',"All these were reckoned by genealogies in the days of Jotham king of Judah, and in the days of Jeroboam king of Israel."],
['1Chronicles','1Chr','05','018','18',"The sons of Reuben, and the Gadites, and half the tribe of Manasseh, of valiant men, men able to bear buckler and sword, and to shoot with bow, and skilful in war, \emph{were} four and forty thousand seven hundred and threescore, that went out to the war."],
['1Chronicles','1Chr','05','019','19',"And they made war with the Hagarites, with Jetur, and Nephish, and Nodab."],
['1Chronicles','1Chr','05','020','20',"And they were helped against them, and the Hagarites were delivered into their hand, and all that \emph{were} with them: for they cried to God in the battle, and he was intreated of them; because they put their trust in him."],
['1Chronicles','1Chr','05','021','21',"And they took away their cattle; of their camels fifty thousand, and of sheep two hundred and fifty thousand, and of asses two thousand, and of men an hundred thousand."],
['1Chronicles','1Chr','05','022','22',"For there fell down many slain, because the war \emph{was} of God. And they dwelt in their steads until the captivity."],
['1Chronicles','1Chr','05','023','23',"And the children of the half tribe of Manasseh dwelt in the land: they increased from Bashan unto Baal-hermon and Senir, and unto mount Hermon."],
['1Chronicles','1Chr','05','024','24',"And these \emph{were} the heads of the house of their fathers, even Epher, and Ishi, and Eliel, and Azriel, and Jeremiah, and Hodaviah, and Jahdiel, mighty men of valour, famous men, \emph{and} heads of the house of their fathers."],
['1Chronicles','1Chr','05','025','25',"And they transgressed against the God of their fathers, and went a whoring after the gods of the people of the land, whom God destroyed before them."],
['1Chronicles','1Chr','05','026','26',"And the God of Israel stirred up the spirit of Pul king of Assyria, and the spirit of Tilgath-pilneser king of Assyria, and he carried them away, even the Reubenites, and the Gadites, and the half tribe of Manasseh, and brought them unto Halah, and Habor, and Hara, and to the river Gozan, unto this day."]
]

FirstChronicles06 =[
    ['1Chronicles','1Chr','06','001','1',"The sons of Levi; Gershon, Kohath, and Merari."],
['1Chronicles','1Chr','06','002','2',"And the sons of Kohath; Amram, Izhar, and Hebron, and Uzziel."],
['1Chronicles','1Chr','06','003','3',"And the children of Amram; Aaron, and Moses, and Miriam. The sons also of Aaron; Nadab, and Abihu, Eleazar, and Ithamar."],
['1Chronicles','1Chr','06','004','4',"Eleazar begat Phinehas, Phinehas begat Abishua,"],
['1Chronicles','1Chr','06','005','5',"And Abishua begat Bukki, and Bukki begat Uzzi,"],
['1Chronicles','1Chr','06','006','6',"And Uzzi begat Zerahiah, and Zerahiah begat Meraioth,"],
['1Chronicles','1Chr','06','007','7',"Meraioth begat Amariah, and Amariah begat Ahitub,"],
['1Chronicles','1Chr','06','008','8',"And Ahitub begat Zadok, and Zadok begat Ahimaaz,"],
['1Chronicles','1Chr','06','009','9',"And Ahimaaz begat Azariah, and Azariah begat Johanan,"],
['1Chronicles','1Chr','06','010','10',"And Johanan begat Azariah, (he \emph{it} \emph{is} that executed the priest's office in the temple that Solomon built in Jerusalem:)"],
['1Chronicles','1Chr','06','011','11',"And Azariah begat Amariah, and Amariah begat Ahitub,"],
['1Chronicles','1Chr','06','012','12',"And Ahitub begat Zadok, and Zadok begat Shallum,"],
['1Chronicles','1Chr','06','013','13',"And Shallum begat Hilkiah, and Hilkiah begat Azariah,"],
['1Chronicles','1Chr','06','014','14',"And Azariah begat Seraiah, and Seraiah begat Jehozadak,"],
['1Chronicles','1Chr','06','015','15',"And Jehozadak went \emph{into} \emph{captivity}, when the LORD carried away Judah and Jerusalem by the hand of Nebuchadnezzar."],
['1Chronicles','1Chr','06','016','16',"The sons of Levi; Gershom, Kohath, and Merari."],
['1Chronicles','1Chr','06','017','17',"And these \emph{be} the names of the sons of Gershom; Libni, and Shimei."],
['1Chronicles','1Chr','06','018','18',"And the sons of Kohath \emph{were}, Amram, and Izhar, and Hebron, and Uzziel."],
['1Chronicles','1Chr','06','019','19',"The sons of Merari; Mahli, and Mushi. And these \emph{are} the families of the Levites according to their fathers."],
['1Chronicles','1Chr','06','020','20',"Of Gershom; Libni his son, Jahath his son, Zimmah his son,"],
['1Chronicles','1Chr','06','021','21',"Joah his son, Iddo his son, Zerah his son, Jeaterai his son."],
['1Chronicles','1Chr','06','022','22',"The sons of Kohath; Amminadab his son, Korah his son, Assir his son,"],
['1Chronicles','1Chr','06','023','23',"Elkanah his son, and Ebiasaph his son, and Assir his son,"],
['1Chronicles','1Chr','06','024','24',"Tahath his son, Uriel his son, Uzziah his son, and Shaul his son."],
['1Chronicles','1Chr','06','025','25',"And the sons of Elkanah; Amasai, and Ahimoth."],
['1Chronicles','1Chr','06','026','26',"\emph{As} \emph{for} Elkanah: the sons of Elkanah; Zophai his son, and Nahath his son,"],
['1Chronicles','1Chr','06','027','27',"Eliab his son, Jeroham his son, Elkanah his son."],
['1Chronicles','1Chr','06','028','28',"And the sons of Samuel; the firstborn Vashni, and Abiah."],
['1Chronicles','1Chr','06','029','29',"The sons of Merari; Mahli, Libni his son, Shimei his son, Uzza his son,"],
['1Chronicles','1Chr','06','030','30',"Shimea his son, Haggiah his son, Asaiah his son."],
['1Chronicles','1Chr','06','031','31',"And these \emph{are} \emph{they} whom David set over the service of song in the house of the LORD, after that the ark had rest."],
['1Chronicles','1Chr','06','032','32',"And they ministered before the dwelling place of the tabernacle of the congregation with singing, until Solomon had built the house of the LORD in Jerusalem: and \emph{then} they waited on their office according to their order."],
['1Chronicles','1Chr','06','033','33',"And these \emph{are} they that waited with their children. Of the sons of the Kohathites: Heman a singer, the son of Joel, the son of Shemuel,"],
['1Chronicles','1Chr','06','034','34',"The son of Elkanah, the son of Jeroham, the son of Eliel, the son of Toah,"],
['1Chronicles','1Chr','06','035','35',"The son of Zuph, the son of Elkanah, the son of Mahath, the son of Amasai,"],
['1Chronicles','1Chr','06','036','36',"The son of Elkanah, the son of Joel, the son of Azariah, the son of Zephaniah,"],
['1Chronicles','1Chr','06','037','37',"The son of Tahath, the son of Assir, the son of Ebiasaph, the son of Korah,"],
['1Chronicles','1Chr','06','038','38',"The son of Izhar, the son of Kohath, the son of Levi, the son of Israel."],
['1Chronicles','1Chr','06','039','39',"And his brother Asaph, who stood on his right hand, \emph{even} Asaph the son of Berachiah, the son of Shimea,"],
['1Chronicles','1Chr','06','040','40',"The son of Michael, the son of Baaseiah, the son of Malchiah,"],
['1Chronicles','1Chr','06','041','41',"The son of Ethni, the son of Zerah, the son of Adaiah,"],
['1Chronicles','1Chr','06','042','42',"The son of Ethan, the son of Zimmah, the son of Shimei,"],
['1Chronicles','1Chr','06','043','43',"The son of Jahath, the son of Gershom, the son of Levi."],
['1Chronicles','1Chr','06','044','44',"And their brethren the sons of Merari \emph{stood} on the left hand: Ethan the son of Kishi, the son of Abdi, the son of Malluch,"],
['1Chronicles','1Chr','06','045','45',"The son of Hashabiah, the son of Amaziah, the son of Hilkiah,"],
['1Chronicles','1Chr','06','046','46',"The son of Amzi, the son of Bani, the son of Shamer,"],
['1Chronicles','1Chr','06','047','47',"The son of Mahli, the son of Mushi, the son of Merari, the son of Levi."],
['1Chronicles','1Chr','06','048','48',"Their brethren also the Levites \emph{were} appointed unto all manner of service of the tabernacle of the house of God."],
['1Chronicles','1Chr','06','049','49',"But Aaron and his sons offered upon the altar of the burnt offering, and on the altar of incense, \emph{and} \emph{were} \emph{appointed} for all the work of the \emph{place} most holy, and to make an atonement for Israel, according to all that Moses the servant of God had commanded."],
['1Chronicles','1Chr','06','050','50',"And these \emph{are} the sons of Aaron; Eleazar his son, Phinehas his son, Abishua his son,"],
['1Chronicles','1Chr','06','051','51',"Bukki his son, Uzzi his son, Zerahiah his son,"],
['1Chronicles','1Chr','06','052','52',"Meraioth his son, Amariah his son, Ahitub his son,"],
['1Chronicles','1Chr','06','053','53',"Zadok his son, Ahimaaz his son."],
['1Chronicles','1Chr','06','054','54',"Now these \emph{are} their dwelling places throughout their castles in their coasts, of the sons of Aaron, of the families of the Kohathites: for their's was the lot."],
['1Chronicles','1Chr','06','055','55',"And they gave them Hebron in the land of Judah, and the suburbs thereof round about it."],
['1Chronicles','1Chr','06','056','56',"But the fields of the city, and the villages thereof, they gave to Caleb the son of Jephunneh."],
['1Chronicles','1Chr','06','057','57',"And to the sons of Aaron they gave the cities of Judah, \emph{namely}, Hebron, \emph{the} \emph{city} of refuge, and Libnah with her suburbs, and Jattir, and Eshtemoa, with their suburbs,"],
['1Chronicles','1Chr','06','058','58',"And Hilen with her suburbs, Debir with her suburbs,"],
['1Chronicles','1Chr','06','059','59',"And Ashan with her suburbs, and Beth-shemesh with her suburbs:"],
['1Chronicles','1Chr','06','060','60',"And out of the tribe of Benjamin; Geba with her suburbs, and Alemeth with her suburbs, and Anathoth with her suburbs. All their cities throughout their families \emph{were} thirteen cities."],
['1Chronicles','1Chr','06','061','61',"And unto the sons of Kohath, \emph{which} \emph{were} left of the family of that tribe, \emph{were} \emph{cities} \emph{given} out of the half tribe, \emph{namely,} \emph{out} \emph{of} the half \emph{tribe} of Manasseh, by lot, ten cities."],
['1Chronicles','1Chr','06','062','62',"And to the sons of Gershom throughout their families out of the tribe of Issachar, and out of the tribe of Asher, and out of the tribe of Naphtali, and out of the tribe of Manasseh in Bashan, thirteen cities."],
['1Chronicles','1Chr','06','063','63',"Unto the sons of Merari \emph{were} \emph{given} by lot, throughout their families, out of the tribe of Reuben, and out of the tribe of Gad, and out of the tribe of Zebulun, twelve cities."],
['1Chronicles','1Chr','06','064','64',"And the children of Israel gave to the Levites \emph{these} cities with their suburbs."],
['1Chronicles','1Chr','06','065','65',"And they gave by lot out of the tribe of the children of Judah, and out of the tribe of the children of Simeon, and out of the tribe of the children of Benjamin, these cities, which are called by \emph{their} names."],
['1Chronicles','1Chr','06','066','66',"And \emph{the} \emph{residue} of the families of the sons of Kohath had cities of their coasts out of the tribe of Ephraim."],
['1Chronicles','1Chr','06','067','67',"And they gave unto them, \emph{of} the cities of refuge, Shechem in mount Ephraim with her suburbs; \emph{they} \emph{gave} also Gezer with her suburbs,"],
['1Chronicles','1Chr','06','068','68',"And Jokmeam with her suburbs, and Beth-horon with her suburbs,"],
['1Chronicles','1Chr','06','069','69',"And Aijalon with her suburbs, and Gath-rimmon with her suburbs:"],
['1Chronicles','1Chr','06','070','70',"And out of the half tribe of Manasseh; Aner with her suburbs, and Bileam with her suburbs, for the family of the remnant of the sons of Kohath."],
['1Chronicles','1Chr','06','071','71',"Unto the sons of Gershom \emph{were} \emph{given} out of the family of the half tribe of Manasseh, Golan in Bashan with her suburbs, and Ashtaroth with her suburbs:"],
['1Chronicles','1Chr','06','072','72',"And out of the tribe of Issachar; Kedesh with her suburbs, Daberath with her suburbs,"],
['1Chronicles','1Chr','06','073','73',"And Ramoth with her suburbs, and Anem with her suburbs:"],
['1Chronicles','1Chr','06','074','74',"And out of the tribe of Asher; Mashal with her suburbs, and Abdon with her suburbs,"],
['1Chronicles','1Chr','06','075','75',"And Hukok with her suburbs, and Rehob with her suburbs:"],
['1Chronicles','1Chr','06','076','76',"And out of the tribe of Naphtali; Kedesh in Galilee with her suburbs, and Hammon with her suburbs, and Kirjathaim with her suburbs."],
['1Chronicles','1Chr','06','077','77',"Unto the rest of the children of Merari \emph{were} \emph{given} out of the tribe of Zebulun, Rimmon with her suburbs, Tabor with her suburbs:"],
['1Chronicles','1Chr','06','078','78',"And on the other side Jordan by Jericho, on the east side of Jordan, \emph{were} \emph{given} \emph{them} out of the tribe of Reuben, Bezer in the wilderness with her suburbs, and Jahzah with her suburbs,"],
['1Chronicles','1Chr','06','079','79',"Kedemoth also with her suburbs, and Mephaath with her suburbs:"],
['1Chronicles','1Chr','06','080','80',"And out of the tribe of Gad; Ramoth in Gilead with her suburbs, and Mahanaim with her suburbs,"],
['1Chronicles','1Chr','06','081','81',"And Heshbon with her suburbs, and Jazer with her suburbs."]
]

FirstChronicles07 = [
    ['1Chronicles','1Chr','07','001','1',"Now the sons of Issachar \emph{were}, Tola, and Puah, Jashub, and Shimron, four."],
['1Chronicles','1Chr','07','002','2',"And the sons of Tola; Uzzi, and Rephaiah, and Jeriel, and Jahmai, and Jibsam, and Shemuel, heads of their father's house, \emph{to} \emph{wit}, of Tola: \emph{they} \emph{were} valiant men of might in their generations; whose number \emph{was} in the days of David two and twenty thousand and six hundred."],
['1Chronicles','1Chr','07','003','3',"And the sons of Uzzi; Izrahiah: and the sons of Izrahiah; Michael, and Obadiah, and Joel, Ishiah, five: all of them chief men."],
['1Chronicles','1Chr','07','004','4',"And with them, by their generations, after the house of their fathers, \emph{were} bands of soldiers for war, six and thirty thousand \emph{men}: for they had many wives and sons."],
['1Chronicles','1Chr','07','005','5',"And their brethren among all the families of Issachar \emph{were} valiant men of might, reckoned in all by their genealogies fourscore and seven thousand."],
['1Chronicles','1Chr','07','006','6',"\emph{The} \emph{sons} of Benjamin; Bela, and Becher, and Jediael, three."],
['1Chronicles','1Chr','07','007','7',"And the sons of Bela; Ezbon, and Uzzi, and Uzziel, and Jerimoth, and Iri, five; heads of the house of \emph{their} fathers, mighty men of valour; and were reckoned by their genealogies twenty and two thousand and thirty and four."],
['1Chronicles','1Chr','07','008','8',"And the sons of Becher; Zemira, and Joash, and Eliezer, and Elioenai, and Omri, and Jerimoth, and Abiah, and Anathoth, and Alameth. All these \emph{are} the sons of Becher."],
['1Chronicles','1Chr','07','009','9',"And the number of them, after their genealogy by their generations, heads of the house of their fathers, mighty men of valour, \emph{was} twenty thousand and two hundred."],
['1Chronicles','1Chr','07','010','10',"The sons also of Jediael; Bilhan: and the sons of Bilhan; Jeush, and Benjamin, and Ehud, and Chenaanah, and Zethan, and Tharshish, and Ahishahar."],
['1Chronicles','1Chr','07','011','11',"All these the sons of Jediael, by the heads of their fathers, mighty men of valour, \emph{were} seventeen thousand and two hundred \emph{soldiers}, fit to go out for war \emph{and} battle."],
['1Chronicles','1Chr','07','012','12',"Shuppim also, and Huppim, the children of Ir, \emph{and} Hushim, the sons of Aher."],
['1Chronicles','1Chr','07','013','13',"The sons of Naphtali; Jahziel, and Guni, and Jezer, and Shallum, the sons of Bilhah."],
['1Chronicles','1Chr','07','014','14',"The sons of Manasseh; Ashriel, whom she bare: \emph{(but} his concubine the Aramitess bare Machir the father of Gilead:"],
['1Chronicles','1Chr','07','015','15',"And Machir took to wife \emph{the} \emph{sister} of Huppim and Shuppim, whose sister's name \emph{was} Maachah;) and the name of the second \emph{was} Zelophehad: and Zelophehad had daughters."],
['1Chronicles','1Chr','07','016','16',"And Maachah the wife of Machir bare a son, and she called his name Peresh; and the name of his brother \emph{was} Sheresh; and his sons \emph{were} Ulam and Rakem."],
['1Chronicles','1Chr','07','017','17',"And the sons of Ulam; Bedan. These \emph{were} the sons of Gilead, the son of Machir, the son of Manasseh."],
['1Chronicles','1Chr','07','018','18',"And his sister Hammoleketh bare Ishod, and Abiezer, and Mahalah."],
['1Chronicles','1Chr','07','019','19',"And the sons of Shemidah were, Ahian, and Shechem, and Likhi, and Aniam."],
['1Chronicles','1Chr','07','020','20',"And the sons of Ephraim; Shuthelah, and Bered his son, and Tahath his son, and Eladah his son, and Tahath his son,"],
['1Chronicles','1Chr','07','021','21',"And Zabad his son, and Shuthelah his son, and Ezer, and Elead, whom the men of Gath \emph{that} \emph{were} born in \emph{that} land slew, because they came down to take away their cattle."],
['1Chronicles','1Chr','07','022','22',"And Ephraim their father mourned many days, and his brethren came to comfort him."],
['1Chronicles','1Chr','07','023','23',"And when he went in to his wife, she conceived, and bare a son, and he called his name Beriah, because it went evil with his house."],
['1Chronicles','1Chr','07','024','24',"(And his daughter \emph{was} Sherah, who built Beth-horon the nether, and the upper, and Uzzen-sherah.)"],
['1Chronicles','1Chr','07','025','25',"And Rephah \emph{was} his son, also Resheph, and Telah his son, and Tahan his son,"],
['1Chronicles','1Chr','07','026','26',"Laadan his son, Ammihud his son, Elishama his son,"],
['1Chronicles','1Chr','07','027','27',"Non his son, Jehoshua his son."],
['1Chronicles','1Chr','07','028','28',"And their possessions and habitations \emph{were}, Beth-el and the towns thereof, and eastward Naaran, and westward Gezer, with the towns thereof; Shechem also and the towns thereof, unto Gaza and the towns thereof:"],
['1Chronicles','1Chr','07','029','29',"And by the borders of the children of Manasseh, Beth-shean and her towns, Taanach and her towns, Megiddo and her towns, Dor and her towns. In these dwelt the children of Joseph the son of Israel."],
['1Chronicles','1Chr','07','030','30',"The sons of Asher; Imnah, and Isuah, and Ishuai, and Beriah, and Serah their sister."],
['1Chronicles','1Chr','07','031','31',"And the sons of Beriah; Heber, and Malchiel, who \emph{is} the father of Birzavith."],
['1Chronicles','1Chr','07','032','32',"And Heber begat Japhlet, and Shomer, and Hotham, and Shua their sister."],
['1Chronicles','1Chr','07','033','33',"And the sons of Japhlet; Pasach, and Bimhal, and Ashvath. These \emph{are} the children of Japhlet."],
['1Chronicles','1Chr','07','034','34',"And the sons of Shamer; Ahi, and Rohgah, Jehubbah, and Aram."],
['1Chronicles','1Chr','07','035','35',"And the sons of his brother Helem; Zophah, and Imna, and Shelesh, and Amal."],
['1Chronicles','1Chr','07','036','36',"The sons of Zophah; Suah, and Harnepher, and Shual, and Beri, and Imrah,"],
['1Chronicles','1Chr','07','037','37',"Bezer, and Hod, and Shamma, and Shilshah, and Ithran, and Beera."],
['1Chronicles','1Chr','07','038','38',"And the sons of Jether; Jephunneh, and Pispah, and Ara."],
['1Chronicles','1Chr','07','039','39',"And the sons of Ulla; Arah, and Haniel, and Rezia."],
['1Chronicles','1Chr','07','040','40',"All these \emph{were} the children of Asher, heads of \emph{their} father's house, choice \emph{and} mighty men of valour, chief of the princes. And the number throughout the genealogy of them that were apt to the war \emph{and} to battle \emph{was} twenty and six thousand men."]
]

FirstChronicles08 = [
    ['1Chronicles','1Chr','08','001','1',"Now Benjamin begat Bela his firstborn, Ashbel the second, and Aharah the third,"],
['1Chronicles','1Chr','08','002','2',"Nohah the fourth, and Rapha the fifth."],
['1Chronicles','1Chr','08','003','3',"And the sons of Bela were, Addar, and Gera, and Abihud,"],
['1Chronicles','1Chr','08','004','4',"And Abishua, and Naaman, and Ahoah,"],
['1Chronicles','1Chr','08','005','5',"And Gera, and Shephuphan, and Huram."],
['1Chronicles','1Chr','08','006','6',"And these \emph{are} the sons of Ehud: these are the heads of the fathers of the inhabitants of Geba, and they removed them to Manahath:"],
['1Chronicles','1Chr','08','007','7',"And Naaman, and Ahiah, and Gera, he removed them, and begat Uzza, and Ahihud."],
['1Chronicles','1Chr','08','008','8',"And Shaharaim begat \emph{children} in the country of Moab, after he had sent them away; Hushim and Baara \emph{were} his wives."],
['1Chronicles','1Chr','08','009','9',"And he begat of Hodesh his wife, Jobab, and Zibia, and Mesha, and Malcham,"],
['1Chronicles','1Chr','08','010','10',"And Jeuz, and Shachia, and Mirma. These \emph{were} his sons, heads of the fathers."],
['1Chronicles','1Chr','08','011','11',"And of Hushim he begat Abitub, and Elpaal."],
['1Chronicles','1Chr','08','012','12',"The sons of Elpaal; Eber, and Misham, and Shamed, who built Ono, and Lod, with the towns thereof:"],
['1Chronicles','1Chr','08','013','13',"Beriah also, and Shema, who \emph{were} heads of the fathers of the inhabitants of Aijalon, who drove away the inhabitants of Gath:"],
['1Chronicles','1Chr','08','014','14',"And Ahio, Shashak, and Jeremoth,"],
['1Chronicles','1Chr','08','015','15',"And Zebadiah, and Arad, and Ader,"],
['1Chronicles','1Chr','08','016','16',"And Michael, and Ispah, and Joha, the sons of Beriah;"],
['1Chronicles','1Chr','08','017','17',"And Zebadiah, and Meshullam, and Hezeki, and Heber,"],
['1Chronicles','1Chr','08','018','18',"Ishmerai also, and Jezliah, and Jobab, the sons of Elpaal;"],
['1Chronicles','1Chr','08','019','19',"And Jakim, and Zichri, and Zabdi,"],
['1Chronicles','1Chr','08','020','20',"And Elienai, and Zilthai, and Eliel,"],
['1Chronicles','1Chr','08','021','21',"And Adaiah, and Beraiah, and Shimrath, the sons of Shimhi;"],
['1Chronicles','1Chr','08','022','22',"And Ishpan, and Heber, and Eliel,"],
['1Chronicles','1Chr','08','023','23',"And Abdon, and Zichri, and Hanan,"],
['1Chronicles','1Chr','08','024','24',"And Hananiah, and Elam, and Antothijah,"],
['1Chronicles','1Chr','08','025','25',"And Iphedeiah, and Penuel, the sons of Shashak;"],
['1Chronicles','1Chr','08','026','26',"And Shamsherai, and Shehariah, and Athaliah,"],
['1Chronicles','1Chr','08','027','27',"And Jaresiah, and Eliah, and Zichri, the sons of Jeroham."],
['1Chronicles','1Chr','08','028','28',"These \emph{were} heads of the fathers, by their generations, chief \emph{men}. These dwelt in Jerusalem."],
['1Chronicles','1Chr','08','029','29',"And at Gibeon dwelt the father of Gibeon; whose wife's name \emph{was} Maachah:"],
['1Chronicles','1Chr','08','030','30',"And his firstborn son Abdon, and Zur, and Kish, and Baal, and Nadab,"],
['1Chronicles','1Chr','08','031','31',"And Gedor, and Ahio, and Zacher."],
['1Chronicles','1Chr','08','032','32',"And Mikloth begat Shimeah. And these also dwelt with their brethren in Jerusalem, over against them."],
['1Chronicles','1Chr','08','033','33',"And Ner begat Kish, and Kish begat Saul, and Saul begat Jonathan, and Malchi-shua, and Abinadab, and Esh-baal."],
['1Chronicles','1Chr','08','034','34',"And the son of Jonathan \emph{was} Merib-baal; and Merib-baal begat Micah."],
['1Chronicles','1Chr','08','035','35',"And the sons of Micah \emph{were}, Pithon, and Melech, and Tarea, and Ahaz."],
['1Chronicles','1Chr','08','036','36',"And Ahaz begat Jehoadah; and Jehoadah begat Alemeth, and Azmaveth, and Zimri; and Zimri begat Moza,"],
['1Chronicles','1Chr','08','037','37',"And Moza begat Binea: Rapha \emph{was} his son, Eleasah his son, Azel his son:"],
['1Chronicles','1Chr','08','038','38',"And Azel had six sons, whose names \emph{are} these, Azrikam, Bocheru, and Ishmael, and Sheariah, and Obadiah, and Hanan. All these \emph{were} the sons of Azel."],
['1Chronicles','1Chr','08','039','39',"And the sons of Eshek his brother \emph{were}, Ulam his firstborn, Jehush the second, and Eliphelet the third."],
['1Chronicles','1Chr','08','040','40',"And the sons of Ulam were mighty men of valour, archers, and had many sons, and sons' sons, an hundred and fifty. All these \emph{are} of the sons of Benjamin."]
]

FirstChronicles09 = [
    ['1Chronicles','1Chr','09','001','1',"And when the queen of Sheba heard of the fame of Solomon, she came to prove Solomon with hard questions at Jerusalem, with a very great company, and camels that bare spices, and gold in abundance, and precious stones: and when she was come to Solomon, she communed with him of all that was in her heart."],
['1Chronicles','1Chr','09','002','2',"And Solomon told her all her questions: and there was nothing hid from Solomon which he told her not."],
['1Chronicles','1Chr','09','003','3',"And when the queen of Sheba had seen the wisdom of Solomon, and the house that he had built,"],
['1Chronicles','1Chr','09','004','4',"And the meat of his table, and the sitting of his servants, and the attendance of his ministers, and their apparel; his cupbearers also, and their apparel; and his ascent by which he went up into the house of the LORD; there was no more spirit in her."],
['1Chronicles','1Chr','09','005','5',"And she said to the king, \emph{It} \emph{was} a true report which I heard in mine own land of thine acts, and of thy wisdom:"],
['1Chronicles','1Chr','09','006','6',"Howbeit I believed not their words, until I came, and mine eyes had seen \emph{it}: and, behold, the one half of the greatness of thy wisdom was not told me: \emph{for} thou exceedest the fame that I heard."],
['1Chronicles','1Chr','09','007','7',"Happy \emph{are} thy men, and happy \emph{are} these thy servants, which stand continually before thee, and hear thy wisdom."],
['1Chronicles','1Chr','09','008','8',"Blessed be the LORD thy God, which delighted in thee to set thee on his throne, \emph{to} \emph{be} king for the LORD thy God: because thy God loved Israel, to establish them for ever, therefore made he thee king over them, to do judgment and justice."],
['1Chronicles','1Chr','09','009','9',"And she gave the king an hundred and twenty talents of gold, and of spices great abundance, and precious stones: neither was there any such spice as the queen of Sheba gave king Solomon."],
['1Chronicles','1Chr','09','010','10',"And the servants also of Huram, and the servants of Solomon, which brought gold from Ophir, brought algum trees and precious stones."],
['1Chronicles','1Chr','09','011','11',"And the king made \emph{of} the algum trees terraces to the house of the LORD, and to the king's palace, and harps and psalteries for singers: and there were none such seen before in the land of Judah."],
['1Chronicles','1Chr','09','012','12',"And king Solomon gave to the queen of Sheba all her desire, whatsoever she asked, beside \emph{that} which she had brought unto the king. So she turned, and went away to her own land, she and her servants."],
['1Chronicles','1Chr','09','013','13',"Now the weight of gold that came to Solomon in one year was six hundred and threescore and six talents of gold;"],
['1Chronicles','1Chr','09','014','14',"Beside \emph{that} \emph{which} chapmen and merchants brought. And all the kings of Arabia and governors of the country brought gold and silver to Solomon."],
['1Chronicles','1Chr','09','015','15',"And king Solomon made two hundred targets \emph{of} beaten gold: six hundred \emph{shekels} of beaten gold went to one target."],
['1Chronicles','1Chr','09','016','16',"And three hundred shields \emph{made} \emph{he} \emph{of} beaten gold: three hundred \emph{shekels} of gold went to one shield. And the king put them in the house of the forest of Lebanon."],
['1Chronicles','1Chr','09','017','17',"Moreover the king made a great throne of ivory, and overlaid it with pure gold."],
['1Chronicles','1Chr','09','018','18',"And \emph{there} \emph{were} six steps to the throne, with a footstool of gold, \emph{which} \emph{were} fastened to the throne, and stays on each side of the sitting place, and two lions standing by the stays:"],
['1Chronicles','1Chr','09','019','19',"And twelve lions stood there on the one side and on the other upon the six steps. There was not the like made in any kingdom."],
['1Chronicles','1Chr','09','020','20',"And all the drinking vessels of king Solomon \emph{were} \emph{of} gold, and all the vessels of the house of the forest of Lebanon \emph{were} \emph{of} pure gold: none \emph{were} \emph{of} silver; it was \emph{not} any thing accounted of in the days of Solomon."],
['1Chronicles','1Chr','09','021','21',"For the king's ships went to Tarshish with the servants of Huram: every three years once came the ships of Tarshish bringing gold, and silver, ivory, and apes, and peacocks."],
['1Chronicles','1Chr','09','022','22',"And king Solomon passed all the kings of the earth in riches and wisdom."],
['1Chronicles','1Chr','09','023','23',"And all the kings of the earth sought the presence of Solomon, to hear his wisdom, that God had put in his heart."],
['1Chronicles','1Chr','09','024','24',"And they brought every man his present, vessels of silver, and vessels of gold, and raiment, harness, and spices, horses, and mules, a rate year by year."],
['1Chronicles','1Chr','09','025','25',"And Solomon had four thousand stalls for horses and chariots, and twelve thousand horsemen; whom he bestowed in the chariot cities, and with the king at Jerusalem."],
['1Chronicles','1Chr','09','026','26',"And he reigned over all the kings from the river even unto the land of the Philistines, and to the border of Egypt."],
['1Chronicles','1Chr','09','027','27',"And the king made silver in Jerusalem as stones, and cedar trees made he as the sycomore trees that \emph{are} in the low plains in abundance."],
['1Chronicles','1Chr','09','028','28',"And they brought unto Solomon horses out of Egypt, and out of all lands."],
['1Chronicles','1Chr','09','029','29',"Now the rest of the acts of Solomon, first and last, \emph{are} they not written in the book of Nathan the prophet, and in the prophecy of Ahijah the Shilonite, and in the visions of Iddo the seer against Jeroboam the son of Nebat?"],
['1Chronicles','1Chr','09','030','30',"And Solomon reigned in Jerusalem over all Israel forty years."],
['1Chronicles','1Chr','09','031','31',"And Solomon slept with his fathers, and he was buried in the city of David his father: and Rehoboam his son reigned in his stead."]
]

FirstChronicles10 = [
['1Chronicles','1Chr','10','001','1',"Now the Philistines fought against Israel; and the men of Israel fled from before the Philistines, and fell down slain in mount Gilboa."],
['1Chronicles','1Chr','10','002','2',"And the Philistines followed hard after Saul, and after his sons; and the Philistines slew Jonathan, and Abinadab, and Malchi-shua, the sons of Saul."],
['1Chronicles','1Chr','10','003','3',"And the battle went sore against Saul, and the archers hit him, and he was wounded of the archers."],
['1Chronicles','1Chr','10','004','4',"Then said Saul to his armourbearer, Draw thy sword, and thrust me through therewith; lest these uncircumcised come and abuse me. But his armourbearer would not; for he was sore afraid. So Saul took a sword, and fell upon it."],
['1Chronicles','1Chr','10','005','5',"And when his armourbearer saw that Saul was dead, he fell likewise on the sword, and died."],
['1Chronicles','1Chr','10','006','6',"So Saul died, and his three sons, and all his house died together."],
['1Chronicles','1Chr','10','007','7',"And when all the men of Israel that \emph{were} in the valley saw that they fled, and that Saul and his sons were dead, then they forsook their cities, and fled: and the Philistines came and dwelt in them."],
['1Chronicles','1Chr','10','008','8',"And it came to pass on the morrow, when the Philistines came to strip the slain, that they found Saul and his sons fallen in mount Gilboa."],
['1Chronicles','1Chr','10','009','9',"And when they had stripped him, they took his head, and his armour, and sent into the land of the Philistines round about, to carry tidings unto their idols, and to the people."],
['1Chronicles','1Chr','10','010','10',"And they put his armour in the house of their gods, and fastened his head in the temple of Dagon."],
['1Chronicles','1Chr','10','011','11',"And when all Jabesh-gilead heard all that the Philistines had done to Saul,"],
['1Chronicles','1Chr','10','012','12',"They arose, all the valiant men, and took away the body of Saul, and the bodies of his sons, and brought them to Jabesh, and buried their bones under the oak in Jabesh, and fasted seven days."],
['1Chronicles','1Chr','10','013','13',"So Saul died for his transgression which he committed against the LORD, \emph{even} against the word of the LORD, which he kept not, and also for asking \emph{counsel} of \emph{one} \emph{that} \emph{had} a familiar spirit, to enquire \emph{of} \emph{it};"],
['1Chronicles','1Chr','10','014','14',"And enquired not of the LORD: therefore he slew him, and turned the kingdom unto David the son of Jesse."]
]

FirstChronicles11 = [
    ['1Chronicles','1Chr','11','001','1',"Then all Israel gathered themselves to David unto Hebron, saying, Behold, we \emph{are} thy bone and thy flesh."],
['1Chronicles','1Chr','11','002','2',"And moreover in time past, even when Saul was king, thou \emph{wast} he that leddest out and broughtest in Israel: and the LORD thy God said unto thee, Thou shalt feed my people Israel, and thou shalt be ruler over my people Israel."],
['1Chronicles','1Chr','11','003','3',"Therefore came all the elders of Israel to the king to Hebron; and David made a covenant with them in Hebron before the LORD; and they anointed David king over Israel, according to the word of the LORD by Samuel."],
['1Chronicles','1Chr','11','004','4',"And David and all Israel went to Jerusalem, which \emph{is} Jebus; where the Jebusites \emph{were}, the inhabitants of the land."],
['1Chronicles','1Chr','11','005','5',"And the inhabitants of Jebus said to David, Thou shalt not come hither. Nevertheless David took the castle of Zion, which \emph{is} the city of David."],
['1Chronicles','1Chr','11','006','6',"And David said, Whosoever smiteth the Jebusites first shall be chief and captain. So Joab the son of Zeruiah went first up, and was chief."],
['1Chronicles','1Chr','11','007','7',"And David dwelt in the castle; therefore they called it the city of David."],
['1Chronicles','1Chr','11','008','8',"And he built the city round about, even from Millo round about: and Joab repaired the rest of the city."],
['1Chronicles','1Chr','11','009','9',"So David waxed greater and greater: for the LORD of hosts \emph{was} with him."],
['1Chronicles','1Chr','11','010','10',"These also \emph{are} the chief of the mighty men whom David had, who strengthened themselves with him in his kingdom, \emph{and} with all Israel, to make him king, according to the word of the LORD concerning Israel."],
['1Chronicles','1Chr','11','011','11',"And this \emph{is} the number of the mighty men whom David had; Jashobeam, an Hachmonite, the chief of the captains: he lifted up his spear against three hundred slain \emph{by} \emph{him} at one time."],
['1Chronicles','1Chr','11','012','12',"And after him \emph{was} Eleazar the son of Dodo, the Ahohite, who \emph{was} \emph{one} of the three mighties."],
['1Chronicles','1Chr','11','013','13',"He was with David at Pas-dammim, and there the Philistines were gathered together to battle, where was a parcel of ground full of barley; and the people fled from before the Philistines."],
['1Chronicles','1Chr','11','014','14',"And they set themselves in the midst of \emph{that} parcel, and delivered it, and slew the Philistines; and the LORD saved \emph{them} by a great deliverance."],
['1Chronicles','1Chr','11','015','15',"Now three of the thirty captains went down to the rock to David, into the cave of Adullam; and the host of the Philistines encamped in the valley of Rephaim."],
['1Chronicles','1Chr','11','016','16',"And David \emph{was} then in the hold, and the Philistines' garrison \emph{was} then at Beth-lehem."],
['1Chronicles','1Chr','11','017','17',"And David longed, and said, Oh that one would give me drink of the water of the well of Beth-lehem, that \emph{is} at the gate!"],
['1Chronicles','1Chr','11','018','18',"And the three brake through the host of the Philistines, and drew water out of the well of Beth-lehem, that \emph{was} by the gate, and took \emph{it}, and brought \emph{it} to David: but David would not drink \emph{of} it, but poured it out to the LORD,"],
['1Chronicles','1Chr','11','019','19',"And said, My God forbid it me, that I should do this thing: shall I drink the blood of these men that have put their lives in jeopardy? for with \emph{the} \emph{jeopardy} \emph{of} their lives they brought it. Therefore he would not drink it. These things did these three mightiest."],
['1Chronicles','1Chr','11','020','20',"And Abishai the brother of Joab, he was chief of the three: for lifting up his spear against three hundred, he slew \emph{them}, and had a name among the three."],
['1Chronicles','1Chr','11','021','21',"Of the three, he was more honourable than the two; for he was their captain: howbeit he attained not to the \emph{first} three."],
['1Chronicles','1Chr','11','022','22',"Benaiah the son of Jehoiada, the son of a valiant man of Kabzeel, who had done many acts; he slew two lionlike men of Moab: also he went down and slew a lion in a pit in a snowy day."],
['1Chronicles','1Chr','11','023','23',"And he slew an Egyptian, a man of \emph{great} stature, five cubits high; and in the Egyptian's hand \emph{was} a spear like a weaver's beam; and he went down to him with a staff, and plucked the spear out of the Egyptian's hand, and slew him with his own spear."],
['1Chronicles','1Chr','11','024','24',"These \emph{things} did Benaiah the son of Jehoiada, and had the name among the three mighties."],
['1Chronicles','1Chr','11','025','25',"Behold, he was honourable among the thirty, but attained not to the \emph{first} three: and David set him over his guard."],
['1Chronicles','1Chr','11','026','26',"Also the valiant men of the armies \emph{were}, Asahel the brother of Joab, Elhanan the son of Dodo of Beth-lehem,"],
['1Chronicles','1Chr','11','027','27',"Shammoth the Harorite, Helez the Pelonite,"],
['1Chronicles','1Chr','11','028','28',"Ira the son of Ikkesh the Tekoite, Abiezer the Antothite,"],
['1Chronicles','1Chr','11','029','29',"Sibbecai the Hushathite, Ilai the Ahohite,"],
['1Chronicles','1Chr','11','030','30',"Maharai the Netophathite, Heled the son of Baanah the Netophathite,"],
['1Chronicles','1Chr','11','031','31',"Ithai the son of Ribai of Gibeah, \emph{that} \emph{pertained} to the children of Benjamin, Benaiah the Pirathonite,"],
['1Chronicles','1Chr','11','032','32',"Hurai of the brooks of Gaash, Abiel the Arbathite,"],
['1Chronicles','1Chr','11','033','33',"Azmaveth the Baharumite, Eliahba the Shaalbonite,"],
['1Chronicles','1Chr','11','034','34',"The sons of Hashem the Gizonite, Jonathan the son of Shage the Hararite,"],
['1Chronicles','1Chr','11','035','35',"Ahiam the son of Sacar the Hararite, Eliphal the son of Ur,"],
['1Chronicles','1Chr','11','036','36',"Hepher the Mecherathite, Ahijah the Pelonite,"],
['1Chronicles','1Chr','11','037','37',"Hezro the Carmelite, Naarai the son of Ezbai,"],
['1Chronicles','1Chr','11','038','38',"Joel the brother of Nathan, Mibhar the son of Haggeri,"],
['1Chronicles','1Chr','11','039','39',"Zelek the Ammonite, Naharai the Berothite, the armourbearer of Joab the son of Zeruiah,"],
['1Chronicles','1Chr','11','040','40',"Ira the Ithrite, Gareb the Ithrite,"],
['1Chronicles','1Chr','11','041','41',"Uriah the Hittite, Zabad the son of Ahlai,"],
['1Chronicles','1Chr','11','042','42',"Adina the son of Shiza the Reubenite, a captain of the Reubenites, and thirty with him,"],
['1Chronicles','1Chr','11','043','43',"Hanan the son of Maachah, and Joshaphat the Mithnite,"],
['1Chronicles','1Chr','11','044','44',"Uzzia the Ashterathite, Shama and Jehiel the sons of Hothan the Aroerite,"],
['1Chronicles','1Chr','11','045','45',"Jediael the son of Shimri, and Joha his brother, the Tizite,"],
['1Chronicles','1Chr','11','046','46',"Eliel the Mahavite, and Jeribai, and Joshaviah, the sons of Elnaam, and Ithmah the Moabite,"],
['1Chronicles','1Chr','11','047','47',"Eliel, and Obed, and Jasiel the Mesobaite."]
]

FirstChronicles12 = [
    ['1Chronicles','1Chr','12','001','1',"Now these \emph{are} they that came to David to Ziklag, while he yet kept himself close because of Saul the son of Kish: and they \emph{were} among the mighty men, helpers of the war."],
['1Chronicles','1Chr','12','002','2',"\emph{They} \emph{were} armed with bows, and could use both the right hand and the left in \emph{hurling} stones and \emph{shooting} arrows out of a bow, \emph{even} of Saul's brethren of Benjamin."],
['1Chronicles','1Chr','12','003','3',"The chief \emph{was} Ahiezer, then Joash, the sons of Shemaah the Gibeathite; and Jeziel, and Pelet, the sons of Azmaveth; and Berachah, and Jehu the Antothite,"],
['1Chronicles','1Chr','12','004','4',"And Ismaiah the Gibeonite, a mighty man among the thirty, and over the thirty; and Jeremiah, and Jahaziel, and Johanan, and Josabad the Gederathite,"],
['1Chronicles','1Chr','12','005','5',"Eluzai, and Jerimoth, and Bealiah, and Shemariah, and Shephatiah the Haruphite,"],
['1Chronicles','1Chr','12','006','6',"Elkanah, and Jesiah, and Azareel, and Joezer, and Jashobeam, the Korhites,"],
['1Chronicles','1Chr','12','007','7',"And Joelah, and Zebadiah, the sons of Jeroham of Gedor."],
['1Chronicles','1Chr','12','008','8',"And of the Gadites there separated themselves unto David into the hold to the wilderness men of might, \emph{and} men of war \emph{fit} for the battle, that could handle shield and buckler, whose faces \emph{were} \emph{like} the faces of lions, and \emph{were} as swift as the roes upon the mountains;"],
['1Chronicles','1Chr','12','009','9',"Ezer the first, Obadiah the second, Eliab the third,"],
['1Chronicles','1Chr','12','010','10',"Mishmannah the fourth, Jeremiah the fifth,"],
['1Chronicles','1Chr','12','011','11',"Attai the sixth, Eliel the seventh,"],
['1Chronicles','1Chr','12','012','12',"Johanan the eighth, Elzabad the ninth,"],
['1Chronicles','1Chr','12','013','13',"Jeremiah the tenth, Machbanai the eleventh."],
['1Chronicles','1Chr','12','014','14',"These \emph{were} of the sons of Gad, captains of the host: one of the least \emph{was} over an hundred, and the greatest over a thousand."],
['1Chronicles','1Chr','12','015','15',"These \emph{are} they that went over Jordan in the first month, when it had overflown all his banks; and they put to flight all \emph{them} of the valleys, \emph{both} toward the east, and toward the west."],
['1Chronicles','1Chr','12','016','16',"And there came of the children of Benjamin and Judah to the hold unto David."],
['1Chronicles','1Chr','12','017','17',"And David went out to meet them, and answered and said unto them, If ye be come peaceably unto me to help me, mine heart shall be knit unto you: but if \emph{ye} \emph{be} \emph{come} to betray me to mine enemies, seeing \emph{there} \emph{is} no wrong in mine hands, the God of our fathers look \emph{thereon}, and rebuke \emph{it}."],
['1Chronicles','1Chr','12','018','18',"Then the spirit came upon Amasai, \emph{who} \emph{was} chief of the captains, \emph{and} \emph{he} \emph{said}, Thine \emph{are} \emph{we}, David, and on thy side, thou son of Jesse: peace, peace \emph{be} unto thee, and peace \emph{be} to thine helpers; for thy God helpeth thee. Then David received them, and made them captains of the band."],
['1Chronicles','1Chr','12','019','19',"And there fell \emph{some} of Manasseh to David, when he came with the Philistines against Saul to battle: but they helped them not: for the lords of the Philistines upon advisement sent him away, saying, He will fall to his master Saul to \emph{the} \emph{jeopardy} \emph{of} our heads."],
['1Chronicles','1Chr','12','020','20',"As he went to Ziklag, there fell to him of Manasseh, Adnah, and Jozabad, and Jediael, and Michael, and Jozabad, and Elihu, and Zilthai, captains of the thousands that \emph{were} of Manasseh."],
['1Chronicles','1Chr','12','021','21',"And they helped David against the band \emph{of} \emph{the} \emph{rovers}: for they \emph{were} all mighty men of valour, and were captains in the host."],
['1Chronicles','1Chr','12','022','22',"For at \emph{that} time day by day there came to David to help him, until \emph{it} \emph{was} a great host, like the host of God."],
['1Chronicles','1Chr','12','023','23',"And these \emph{are} the numbers of the bands \emph{that} \emph{were} ready armed to the war, \emph{and} came to David to Hebron, to turn the kingdom of Saul to him, according to the word of the LORD."],
['1Chronicles','1Chr','12','024','24',"The children of Judah that bare shield and spear \emph{were} six thousand and eight hundred, ready armed to the war."],
['1Chronicles','1Chr','12','025','25',"Of the children of Simeon, mighty men of valour for the war, seven thousand and one hundred."],
['1Chronicles','1Chr','12','026','26',"Of the children of Levi four thousand and six hundred."],
['1Chronicles','1Chr','12','027','27',"And Jehoiada \emph{was} the leader of the Aaronites, and with him \emph{were} three thousand and seven hundred;"],
['1Chronicles','1Chr','12','028','28',"And Zadok, a young man mighty of valour, and of his father's house twenty and two captains."],
['1Chronicles','1Chr','12','029','29',"And of the children of Benjamin, the kindred of Saul, three thousand: for hitherto the greatest part of them had kept the ward of the house of Saul."],
['1Chronicles','1Chr','12','030','30',"And of the children of Ephraim twenty thousand and eight hundred, mighty men of valour, famous throughout the house of their fathers."],
['1Chronicles','1Chr','12','031','31',"And of the half tribe of Manasseh eighteen thousand, which were expressed by name, to come and make David king."],
['1Chronicles','1Chr','12','032','32',"And of the children of Issachar, \emph{which} \emph{were} \emph{men} that had understanding of the times, to know what Israel ought to do; the heads of them \emph{were} two hundred; and all their brethren \emph{were} at their commandment."],
['1Chronicles','1Chr','12','033','33',"Of Zebulun, such as went forth to battle, expert in war, with all instruments of war, fifty thousand, which could keep rank: \emph{they} \emph{were} not of double heart."],
['1Chronicles','1Chr','12','034','34',"And of Naphtali a thousand captains, and with them with shield and spear thirty and seven thousand."],
['1Chronicles','1Chr','12','035','35',"And of the Danites expert in war twenty and eight thousand and six hundred."],
['1Chronicles','1Chr','12','036','36',"And of Asher, such as went forth to battle, expert in war, forty thousand."],
['1Chronicles','1Chr','12','037','37',"And on the other side of Jordan, of the Reubenites, and the Gadites, and of the half tribe of Manasseh, with all manner of instruments of war for the battle, an hundred and twenty thousand."],
['1Chronicles','1Chr','12','038','38',"All these men of war, that could keep rank, came with a perfect heart to Hebron, to make David king over all Israel: and all the rest also of Israel \emph{were} of one heart to make David king."],
['1Chronicles','1Chr','12','039','39',"And there they were with David three days, eating and drinking: for their brethren had prepared for them."],
['1Chronicles','1Chr','12','040','40',"Moreover they that were nigh them, \emph{even} unto Issachar and Zebulun and Naphtali, brought bread on asses, and on camels, and on mules, and on oxen, \emph{and} meat, meal, cakes of figs, and bunches of raisins, and wine, and oil, and oxen, and sheep abundantly: for \emph{there} \emph{was} joy in Israel."]
]

FirstChronicles13 = [
    ['1Chronicles','1Chr','13','001','1',"And David consulted with the captains of thousands and hundreds, \emph{and} with every leader."],
['1Chronicles','1Chr','13','002','2',"And David said unto all the congregation of Israel, If \emph{it} \emph{seem} good unto you, and \emph{that} \emph{it} \emph{be} of the LORD our God, let us send abroad unto our brethren every where, \emph{that} \emph{are} left in all the land of Israel, and with them \emph{also} to the priests and Levites \emph{which} \emph{are} in their cities \emph{and} suburbs, that they may gather themselves unto us:"],
['1Chronicles','1Chr','13','003','3',"And let us bring again the ark of our God to us: for we enquired not at it in the days of Saul."],
['1Chronicles','1Chr','13','004','4',"And all the congregation said that they would do so: for the thing was right in the eyes of all the people."],
['1Chronicles','1Chr','13','005','5',"So David gathered all Israel together, from Shihor of Egypt even unto the entering of Hemath, to bring the ark of God from Kirjath-jearim."],
['1Chronicles','1Chr','13','006','6',"And David went up, and all Israel, to Baalah, \emph{that} \emph{is}, to Kirjath-jearim, which \emph{belonged} to Judah, to bring up thence the ark of God the LORD, that dwelleth \emph{between} the cherubims, whose name is called \emph{on} \emph{it}."],
['1Chronicles','1Chr','13','007','7',"And they carried the ark of God in a new cart out of the house of Abinadab: and Uzza and Ahio drave the cart."],
['1Chronicles','1Chr','13','008','8',"And David and all Israel played before God with all \emph{their} might, and with singing, and with harps, and with psalteries, and with timbrels, and with cymbals, and with trumpets."],
['1Chronicles','1Chr','13','009','9',"And when they came unto the threshingfloor of Chidon, Uzza put forth his hand to hold the ark; for the oxen stumbled."],
['1Chronicles','1Chr','13','010','10',"And the anger of the LORD was kindled against Uzza, and he smote him, because he put his hand to the ark: and there he died before God."],
['1Chronicles','1Chr','13','011','11',"And David was displeased, because the LORD had made a breach upon Uzza: wherefore that place is called Perez-uzza to this day."],
['1Chronicles','1Chr','13','012','12',"And David was afraid of God that day, saying, How shall I bring the ark of God \emph{home} to me?"],
['1Chronicles','1Chr','13','013','13',"So David brought not the ark \emph{home} to himself to the city of David, but carried it aside into the house of Obed-edom the Gittite."],
['1Chronicles','1Chr','13','014','14',"And the ark of God remained with the family of Obed-edom in his house three months. And the LORD blessed the house of Obed-edom, and all that he had."]
]

FirstChronicles14 = [
    ['1Chronicles','1Chr','14','001','1',"Now Hiram king of Tyre sent messengers to David, and timber of cedars, with masons and carpenters, to build him an house."],
['1Chronicles','1Chr','14','002','2',"And David perceived that the LORD had confirmed him king over Israel, for his kingdom was lifted up on high, because of his people Israel."],
['1Chronicles','1Chr','14','003','3',"And David took more wives at Jerusalem: and David begat more sons and daughters."],
['1Chronicles','1Chr','14','004','4',"Now these \emph{are} the names of \emph{his} children which he had in Jerusalem; Shammua, and Shobab, Nathan, and Solomon,"],
['1Chronicles','1Chr','14','005','5',"And Ibhar, and Elishua, and Elpalet,"],
['1Chronicles','1Chr','14','006','6',"And Nogah, and Nepheg, and Japhia,"],
['1Chronicles','1Chr','14','007','7',"And Elishama, and Beeliada, and Eliphalet."],
['1Chronicles','1Chr','14','008','8',"And when the Philistines heard that David was anointed king over all Israel, all the Philistines went up to seek David. And David heard \emph{of} \emph{it}, and went out against them."],
['1Chronicles','1Chr','14','009','9',"And the Philistines came and spread themselves in the valley of Rephaim."],
['1Chronicles','1Chr','14','010','10',"And David enquired of God, saying, Shall I go up against the Philistines? and wilt thou deliver them into mine hand? And the LORD said unto him, Go up; for I will deliver them into thine hand."],
['1Chronicles','1Chr','14','011','11',"So they came up to Baal-perazim; and David smote them there. Then David said, God hath broken in upon mine enemies by mine hand like the breaking forth of waters: therefore they called the name of that place Baal-perazim."],
['1Chronicles','1Chr','14','012','12',"And when they had left their gods there, David gave a commandment, and they were burned with fire."],
['1Chronicles','1Chr','14','013','13',"And the Philistines yet again spread themselves abroad in the valley."],
['1Chronicles','1Chr','14','014','14',"Therefore David enquired again of God; and God said unto him, Go not up after them; turn away from them, and come upon them over against the mulberry trees."],
['1Chronicles','1Chr','14','015','15',"And it shall be, when thou shalt hear a sound of going in the tops of the mulberry trees, \emph{that} then thou shalt go out to battle: for God is gone forth before thee to smite the host of the Philistines."],
['1Chronicles','1Chr','14','016','16',"David therefore did as God commanded him: and they smote the host of the Philistines from Gibeon even to Gazer."],
['1Chronicles','1Chr','14','017','17',"And the fame of David went out into all lands; and the LORD brought the fear of him upon all nations."]
]

FirstChronicles15 = [
    ['1Chronicles','1Chr','15','001','1',"And \emph{David} made him houses in the city of David, and prepared a place for the ark of God, and pitched for it a tent."],
['1Chronicles','1Chr','15','002','2',"Then David said, None ought to carry the ark of God but the Levites: for them hath the LORD chosen to carry the ark of God, and to minister unto him for ever."],
['1Chronicles','1Chr','15','003','3',"And David gathered all Israel together to Jerusalem, to bring up the ark of the LORD unto his place, which he had prepared for it."],
['1Chronicles','1Chr','15','004','4',"And David assembled the children of Aaron, and the Levites:"],
['1Chronicles','1Chr','15','005','5',"Of the sons of Kohath; Uriel the chief, and his brethren an hundred and twenty:"],
['1Chronicles','1Chr','15','006','6',"Of the sons of Merari; Asaiah the chief, and his brethren two hundred and twenty:"],
['1Chronicles','1Chr','15','007','7',"Of the sons of Gershom; Joel the chief, and his brethren an hundred and thirty:"],
['1Chronicles','1Chr','15','008','8',"Of the sons of Elizaphan; Shemaiah the chief, and his brethren two hundred:"],
['1Chronicles','1Chr','15','009','9',"Of the sons of Hebron; Eliel the chief, and his brethren fourscore:"],
['1Chronicles','1Chr','15','010','10',"Of the sons of Uzziel; Amminadab the chief, and his brethren an hundred and twelve."],
['1Chronicles','1Chr','15','011','11',"And David called for Zadok and Abiathar the priests, and for the Levites, for Uriel, Asaiah, and Joel, Shemaiah, and Eliel, and Amminadab,"],
['1Chronicles','1Chr','15','012','12',"And said unto them, Ye \emph{are} the chief of the fathers of the Levites: sanctify yourselves, \emph{both} ye and your brethren, that ye may bring up the ark of the LORD God of Israel unto \emph{the} \emph{place} \emph{that} I have prepared for it."],
['1Chronicles','1Chr','15','013','13',"For because ye \emph{did} \emph{it} not at the first, the LORD our God made a breach upon us, for that we sought him not after the due order."],
['1Chronicles','1Chr','15','014','14',"So the priests and the Levites sanctified themselves to bring up the ark of the LORD God of Israel."],
['1Chronicles','1Chr','15','015','15',"And the children of the Levites bare the ark of God upon their shoulders with the staves thereon, as Moses commanded according to the word of the LORD."],
['1Chronicles','1Chr','15','016','16',"And David spake to the chief of the Levites to appoint their brethren \emph{to} \emph{be} the singers with instruments of musick, psalteries and harps and cymbals, sounding, by lifting up the voice with joy."],
['1Chronicles','1Chr','15','017','17',"So the Levites appointed Heman the son of Joel; and of his brethren, Asaph the son of Berechiah; and of the sons of Merari their brethren, Ethan the son of Kushaiah;"],
['1Chronicles','1Chr','15','018','18',"And with them their brethren of the second \emph{degree}, Zechariah, Ben, and Jaaziel, and Shemiramoth, and Jehiel, and Unni, Eliab, and Benaiah, and Maaseiah, and Mattithiah, and Elipheleh, and Mikneiah, and Obed-edom, and Jeiel, the porters."],
['1Chronicles','1Chr','15','019','19',"So the singers, Heman, Asaph, and Ethan, \emph{were} \emph{appointed} to sound with cymbals of brass;"],
['1Chronicles','1Chr','15','020','20',"And Zechariah, and Aziel, and Shemiramoth, and Jehiel, and Unni, and Eliab, and Maaseiah, and Benaiah, with psalteries on Alamoth;"],
['1Chronicles','1Chr','15','021','21',"And Mattithiah, and Elipheleh, and Mikneiah, and Obed-edom, and Jeiel, and Azaziah, with harps on the Sheminith to excel."],
['1Chronicles','1Chr','15','022','22',"And Chenaniah, chief of the Levites, \emph{was} for song: he instructed about the song, because he \emph{was} skilful."],
['1Chronicles','1Chr','15','023','23',"And Berechiah and Elkanah \emph{were} doorkeepers for the ark."],
['1Chronicles','1Chr','15','024','24',"And Shebaniah, and Jehoshaphat, and Nethaneel, and Amasai, and Zechariah, and Benaiah, and Eliezer, the priests, did blow with the trumpets before the ark of God: and Obed-edom and Jehiah \emph{were} doorkeepers for the ark."],
['1Chronicles','1Chr','15','025','25',"So David, and the elders of Israel, and the captains over thousands, went to bring up the ark of the covenant of the LORD out of the house of Obed-edom with joy."],
['1Chronicles','1Chr','15','026','26',"And it came to pass, when God helped the Levites that bare the ark of the covenant of the LORD, that they offered seven bullocks and seven rams."],
['1Chronicles','1Chr','15','027','27',"And David \emph{was} clothed with a robe of fine linen, and all the Levites that bare the ark, and the singers, and Chenaniah the master of the song with the singers: David also \emph{had} upon him an ephod of linen."],
['1Chronicles','1Chr','15','028','28',"Thus all Israel brought up the ark of the covenant of the LORD with shouting, and with sound of the cornet, and with trumpets, and with cymbals, making a noise with psalteries and harps."],
['1Chronicles','1Chr','15','029','29',"And it came to pass, \emph{as} the ark of the covenant of the LORD came to the city of David, that Michal the daughter of Saul looking out at a window saw king David dancing and playing: and she despised him in her heart."]
]

FirstChronicles16 = [
    ['1Chronicles','1Chr','16','001','1',"So they brought the ark of God, and set it in the midst of the tent that David had pitched for it: and they offered burnt sacrifices and peace offerings before God."],
['1Chronicles','1Chr','16','002','2',"And when David had made an end of offering the burnt offerings and the peace offerings, he blessed the people in the name of the LORD."],
['1Chronicles','1Chr','16','003','3',"And he dealt to every one of Israel, both man and woman, to every one a loaf of bread, and a good piece of flesh, and a flagon \emph{of} \emph{wine}."],
['1Chronicles','1Chr','16','004','4',"And he appointed \emph{certain} of the Levites to minister before the ark of the LORD, and to record, and to thank and praise the LORD God of Israel:"],
['1Chronicles','1Chr','16','005','5',"Asaph the chief, and next to him Zechariah, Jeiel, and Shemiramoth, and Jehiel, and Mattithiah, and Eliab, and Benaiah, and Obed-edom: and Jeiel with psalteries and with harps; but Asaph made a sound with cymbals;"],
['1Chronicles','1Chr','16','006','6',"Benaiah also and Jahaziel the priests with trumpets continually before the ark of the covenant of God."],
['1Chronicles','1Chr','16','007','7',"Then on that day David delivered first \emph{this} \emph{psalm} to thank the LORD into the hand of Asaph and his brethren."],
['1Chronicles','1Chr','16','008','8',"Give thanks unto the LORD, call upon his name, make known his deeds among the people."],
['1Chronicles','1Chr','16','009','9',"Sing unto him, sing psalms unto him, talk ye of all his wondrous works."],
['1Chronicles','1Chr','16','010','10',"Glory ye in his holy name: let the heart of them rejoice that seek the LORD."],
['1Chronicles','1Chr','16','011','11',"Seek the LORD and his strength, seek his face continually."],
['1Chronicles','1Chr','16','012','12',"Remember his marvellous works that he hath done, his wonders, and the judgments of his mouth;"],
['1Chronicles','1Chr','16','013','13',"O ye seed of Israel his servant, ye children of Jacob, his chosen ones."],
['1Chronicles','1Chr','16','014','14',"He \emph{is} the LORD our God; his judgments \emph{are} in all the earth."],
['1Chronicles','1Chr','16','015','15',"Be ye mindful always of his covenant; the word \emph{which} he commanded to a thousand generations;"],
['1Chronicles','1Chr','16','016','16',"\emph{Even} \emph{of} \emph{the} \emph{covenant} which he made with Abraham, and of his oath unto Isaac;"],
['1Chronicles','1Chr','16','017','17',"And hath confirmed the same to Jacob for a law, \emph{and} to Israel \emph{for} an everlasting covenant,"],
['1Chronicles','1Chr','16','018','18',"Saying, Unto thee will I give the land of Canaan, the lot of your inheritance;"],
['1Chronicles','1Chr','16','019','19',"When ye were but few, even a few, and strangers in it."],
['1Chronicles','1Chr','16','020','20',"And \emph{when} they went from nation to nation, and from \emph{one} kingdom to another people;"],
['1Chronicles','1Chr','16','021','21',"He suffered no man to do them wrong: yea, he reproved kings for their sakes,"],
['1Chronicles','1Chr','16','022','22',"\emph{Saying}, Touch not mine anointed, and do my prophets no harm."],
['1Chronicles','1Chr','16','023','23',"Sing unto the LORD, all the earth; shew forth from day to day his salvation."],
['1Chronicles','1Chr','16','024','24',"Declare his glory among the heathen; his marvellous works among all nations."],
['1Chronicles','1Chr','16','025','25',"For great \emph{is} the LORD, and greatly to be praised: he also \emph{is} to be feared above all gods."],
['1Chronicles','1Chr','16','026','26',"For all the gods of the people \emph{are} idols: but the LORD made the heavens."],
['1Chronicles','1Chr','16','027','27',"Glory and honour \emph{are} in his presence; strength and gladness \emph{are} in his place."],
['1Chronicles','1Chr','16','028','28',"Give unto the LORD, ye kindreds of the people, give unto the LORD glory and strength."],
['1Chronicles','1Chr','16','029','29',"Give unto the LORD the glory \emph{due} unto his name: bring an offering, and come before him: worship the LORD in the beauty of holiness."],
['1Chronicles','1Chr','16','030','30',"Fear before him, all the earth: the world also shall be stable, that it be not moved."],
['1Chronicles','1Chr','16','031','31',"Let the heavens be glad, and let the earth rejoice: and let \emph{men} say among the nations, The LORD reigneth."],
['1Chronicles','1Chr','16','032','32',"Let the sea roar, and the fulness thereof: let the fields rejoice, and all that \emph{is} therein."],
['1Chronicles','1Chr','16','033','33',"Then shall the trees of the wood sing out at the presence of the LORD, because he cometh to judge the earth."],
['1Chronicles','1Chr','16','034','34',"O give thanks unto the LORD; for \emph{he} \emph{is} good; for his mercy \emph{endureth} for ever."],
['1Chronicles','1Chr','16','035','35',"And say ye, Save us, O God of our salvation, and gather us together, and deliver us from the heathen, that we may give thanks to thy holy name, \emph{and} glory in thy praise."],
['1Chronicles','1Chr','16','036','36',"Blessed \emph{be} the LORD God of Israel for ever and ever. And all the people said, Amen, and praised the LORD."],
['1Chronicles','1Chr','16','037','37',"So he left there before the ark of the covenant of the LORD Asaph and his brethren, to minister before the ark continually, as every day's work required:"],
['1Chronicles','1Chr','16','038','38',"And Obed-edom with their brethren, threescore and eight; Obed-edom also the son of Jeduthun and Hosah \emph{to} \emph{be} porters:"],
['1Chronicles','1Chr','16','039','39',"And Zadok the priest, and his brethren the priests, before the tabernacle of the LORD in the high place that \emph{was} at Gibeon,"],
['1Chronicles','1Chr','16','040','40',"To offer burnt offerings unto the LORD upon the altar of the burnt offering continually morning and evening, and \emph{to} \emph{do} according to all that is written in the law of the LORD, which he commanded Israel;"],
['1Chronicles','1Chr','16','041','41',"And with them Heman and Jeduthun, and the rest that were chosen, who were expressed by name, to give thanks to the LORD, because his mercy \emph{endureth} for ever;"],
['1Chronicles','1Chr','16','042','42',"And with them Heman and Jeduthun with trumpets and cymbals for those that should make a sound, and with musical instruments of God. And the sons of Jeduthun \emph{were} porters."],
['1Chronicles','1Chr','16','043','43',"And all the people departed every man to his house: and David returned to bless his house."]
]

FirstChronicles17 = [
    ['1Chronicles','1Chr','17','001','1',"Now it came to pass, as David sat in his house, that David said to Nathan the prophet, Lo, I dwell in an house of cedars, but the ark of the covenant of the LORD \emph{remaineth} under curtains."],
['1Chronicles','1Chr','17','002','2',"Then Nathan said unto David, Do all that \emph{is} in thine heart; for God \emph{is} with thee."],
['1Chronicles','1Chr','17','003','3',"And it came to pass the same night, that the word of God came to Nathan, saying,"],
['1Chronicles','1Chr','17','004','4',"Go and tell David my servant, Thus saith the LORD, Thou shalt not build me an house to dwell in:"],
['1Chronicles','1Chr','17','005','5',"For I have not dwelt in an house since the day that I brought up Israel unto this day; but have gone from tent to tent, and from \emph{one} tabernacle \emph{to} \emph{another}."],
['1Chronicles','1Chr','17','006','6',"Wheresoever I have walked with all Israel, spake I a word to any of the judges of Israel, whom I commanded to feed my people, saying, Why have ye not built me an house of cedars?"],
['1Chronicles','1Chr','17','007','7',"Now therefore thus shalt thou say unto my servant David, Thus saith the LORD of hosts, I took thee from the sheepcote, \emph{even} from following the sheep, that thou shouldest be ruler over my people Israel:"],
['1Chronicles','1Chr','17','008','8',"And I have been with thee whithersoever thou hast walked, and have cut off all thine enemies from before thee, and have made thee a name like the name of the great men that \emph{are} in the earth."],
['1Chronicles','1Chr','17','009','9',"Also I will ordain a place for my people Israel, and will plant them, and they shall dwell in their place, and shall be moved no more; neither shall the children of wickedness waste them any more, as at the beginning,"],
['1Chronicles','1Chr','17','010','10',"And since the time that I commanded judges \emph{to} \emph{be} over my people Israel. Moreover I will subdue all thine enemies. Furthermore I tell thee that the LORD will build thee an house."],
['1Chronicles','1Chr','17','011','11',"And it shall come to pass, when thy days be expired that thou must go \emph{to} \emph{be} with thy fathers, that I will raise up thy seed after thee, which shall be of thy sons; and I will establish his kingdom."],
['1Chronicles','1Chr','17','012','12',"He shall build me an house, and I will stablish his throne for ever."],
['1Chronicles','1Chr','17','013','13',"I will be his father, and he shall be my son: and I will not take my mercy away from him, as I took \emph{it} from \emph{him} that was before thee:"],
['1Chronicles','1Chr','17','014','14',"But I will settle him in mine house and in my kingdom for ever: and his throne shall be established for evermore."],
['1Chronicles','1Chr','17','015','15',"According to all these words, and according to all this vision, so did Nathan speak unto David."],
['1Chronicles','1Chr','17','016','16',"And David the king came and sat before the LORD, and said, Who \emph{am} I, O LORD God, and what \emph{is} mine house, that thou hast brought me hitherto?"],
['1Chronicles','1Chr','17','017','17',"And \emph{yet} this was a small thing in thine eyes, O God; for thou hast \emph{also} spoken of thy servant's house for a great while to come, and hast regarded me according to the estate of a man of high degree, O LORD God."],
['1Chronicles','1Chr','17','018','18',"What can David \emph{speak} more to thee for the honour of thy servant? for thou knowest thy servant."],
['1Chronicles','1Chr','17','019','19',"O LORD, for thy servant's sake, and according to thine own heart, hast thou done all this greatness, in making known all \emph{these} great things."],
['1Chronicles','1Chr','17','020','20',"O LORD, \emph{there} \emph{is} none like thee, neither \emph{is} \emph{there} \emph{any} God beside thee, according to all that we have heard with our ears."],
['1Chronicles','1Chr','17','021','21',"And what one nation in the earth \emph{is} like thy people Israel, whom God went to redeem \emph{to} \emph{be} his own people, to make thee a name of greatness and terribleness, by driving out nations from before thy people, whom thou hast redeemed out of Egypt?"],
['1Chronicles','1Chr','17','022','22',"For thy people Israel didst thou make thine own people for ever; and thou, LORD, becamest their God."],
['1Chronicles','1Chr','17','023','23',"Therefore now, LORD, let the thing that thou hast spoken concerning thy servant and concerning his house be established for ever, and do as thou hast said."],
['1Chronicles','1Chr','17','024','24',"Let it even be established, that thy name may be magnified for ever, saying, The LORD of hosts \emph{is} the God of Israel, \emph{even} a God to Israel: and \emph{let} the house of David thy servant \emph{be} established before thee."],
['1Chronicles','1Chr','17','025','25',"For thou, O my God, hast told thy servant that thou wilt build him an house: therefore thy servant hath found \emph{in} \emph{his} \emph{heart} to pray before thee."],
['1Chronicles','1Chr','17','026','26',"And now, LORD, thou art God, and hast promised this goodness unto thy servant:"],
['1Chronicles','1Chr','17','027','27',"Now therefore let it please thee to bless the house of thy servant, that it may be before thee for ever: for thou blessest, O LORD, and \emph{it} \emph{shall} \emph{be} blessed for ever."]
]

FirstChronicles18 = [
    ['1Chronicles','1Chr','18','001','1',"Now after this it came to pass, that David smote the Philistines, and subdued them, and took Gath and her towns out of the hand of the Philistines."],
['1Chronicles','1Chr','18','002','2',"And he smote Moab; and the Moabites became David's servants, \emph{and} brought gifts."],
['1Chronicles','1Chr','18','003','3',"And David smote Hadarezer king of Zobah unto Hamath, as he went to stablish his dominion by the river Euphrates."],
['1Chronicles','1Chr','18','004','4',"And David took from him a thousand chariots, and seven thousand horsemen, and twenty thousand footmen: David also houghed all the chariot \emph{horses}, but reserved of them an hundred chariots."],
['1Chronicles','1Chr','18','005','5',"And when the Syrians of Damascus came to help Hadarezer king of Zobah, David slew of the Syrians two and twenty thousand men."],
['1Chronicles','1Chr','18','006','6',"Then David put \emph{garrisons} in Syria-damascus; and the Syrians became David's servants, \emph{and} brought gifts. Thus the LORD preserved David whithersoever he went."],
['1Chronicles','1Chr','18','007','7',"And David took the shields of gold that were on the servants of Hadarezer, and brought them to Jerusalem."],
['1Chronicles','1Chr','18','008','8',"Likewise from Tibhath, and from Chun, cities of Hadarezer, brought David very much brass, wherewith Solomon made the brasen sea, and the pillars, and the vessels of brass."],
['1Chronicles','1Chr','18','009','9',"Now when Tou king of Hamath heard how David had smitten all the host of Hadarezer king of Zobah;"],
['1Chronicles','1Chr','18','010','10',"He sent Hadoram his son to king David, to enquire of his welfare, and to congratulate him, because he had fought against Hadarezer, and smitten him; (for Hadarezer had war with Tou;) and \emph{with} \emph{him} all manner of vessels of gold and silver and brass."],
['1Chronicles','1Chr','18','011','11',"Them also king David dedicated unto the LORD, with the silver and the gold that he brought from all \emph{these} nations; from Edom, and from Moab, and from the children of Ammon, and from the Philistines, and from Amalek."],
['1Chronicles','1Chr','18','012','12',"Moreover Abishai the son of Zeruiah slew of the Edomites in the valley of salt eighteen thousand."],
['1Chronicles','1Chr','18','013','13',"And he put garrisons in Edom; and all the Edomites became David's servants. Thus the LORD preserved David whithersoever he went."],
['1Chronicles','1Chr','18','014','14',"So David reigned over all Israel, and executed judgment and justice among all his people."],
['1Chronicles','1Chr','18','015','15',"And Joab the son of Zeruiah \emph{was} over the host; and Jehoshaphat the son of Ahilud, recorder."],
['1Chronicles','1Chr','18','016','16',"And Zadok the son of Ahitub, and Abimelech the son of Abiathar, \emph{were} the priests; and Shavsha was scribe;"],
['1Chronicles','1Chr','18','017','17',"And Benaiah the son of Jehoiada \emph{was} over the Cherethites and the Pelethites; and the sons of David \emph{were} chief about the king."]
]

FirstChronicles19 = [
    ['1Chronicles','1Chr','19','001','1',"Now it came to pass after this, that Nahash the king of the children of Ammon died, and his son reigned in his stead."],
['1Chronicles','1Chr','19','002','2',"And David said, I will shew kindness unto Hanun the son of Nahash, because his father shewed kindness to me. And David sent messengers to comfort him concerning his father. So the servants of David came into the land of the children of Ammon to Hanun, to comfort him."],
['1Chronicles','1Chr','19','003','3',"But the princes of the children of Ammon said to Hanun, Thinkest thou that David doth honour thy father, that he hath sent comforters unto thee? are not his servants come unto thee for to search, and to overthrow, and to spy out the land?"],
['1Chronicles','1Chr','19','004','4',"Wherefore Hanun took David's servants, and shaved them, and cut off their garments in the midst hard by their buttocks, and sent them away."],
['1Chronicles','1Chr','19','005','5',"Then there went \emph{certain}, and told David how the men were served. And he sent to meet them: for the men were greatly ashamed. And the king said, Tarry at Jericho until your beards be grown, and \emph{then} return."],
['1Chronicles','1Chr','19','006','6',"And when the children of Ammon saw that they had made themselves odious to David, Hanun and the children of Ammon sent a thousand talents of silver to hire them chariots and horsemen out of Mesopotamia, and out of Syria-maachah, and out of Zobah."],
['1Chronicles','1Chr','19','007','7',"So they hired thirty and two thousand chariots, and the king of Maachah and his people; who came and pitched before Medeba. And the children of Ammon gathered themselves together from their cities, and came to battle."],
['1Chronicles','1Chr','19','008','8',"And when David heard \emph{of} \emph{it}, he sent Joab, and all the host of the mighty men."],
['1Chronicles','1Chr','19','009','9',"And the children of Ammon came out, and put the battle in array before the gate of the city: and the kings that were come \emph{were} by themselves in the field."],
['1Chronicles','1Chr','19','010','10',"Now when Joab saw that the battle was set against him before and behind, he chose out of all the choice of Israel, and put \emph{them} in array against the Syrians."],
['1Chronicles','1Chr','19','011','11',"And the rest of the people he delivered unto the hand of Abishai his brother, and they set \emph{themselves} in array against the children of Ammon."],
['1Chronicles','1Chr','19','012','12',"And he said, If the Syrians be too strong for me, then thou shalt help me: but if the children of Ammon be too strong for thee, then I will help thee."],
['1Chronicles','1Chr','19','013','13',"Be of good courage, and let us behave ourselves valiantly for our people, and for the cities of our God: and let the LORD do \emph{that} \emph{which} \emph{is} good in his sight."],
['1Chronicles','1Chr','19','014','14',"So Joab and the people that \emph{were} with him drew nigh before the Syrians unto the battle; and they fled before him."],
['1Chronicles','1Chr','19','015','15',"And when the children of Ammon saw that the Syrians were fled, they likewise fled before Abishai his brother, and entered into the city. Then Joab came to Jerusalem."],
['1Chronicles','1Chr','19','016','16',"And when the Syrians saw that they were put to the worse before Israel, they sent messengers, and drew forth the Syrians that \emph{were} beyond the river: and Shophach the captain of the host of Hadarezer \emph{went} before them."],
['1Chronicles','1Chr','19','017','17',"And it was told David; and he gathered all Israel, and passed over Jordan, and came upon them, and set \emph{the} \emph{battle} in array against them. So when David had put the battle in array against the Syrians, they fought with him."],
['1Chronicles','1Chr','19','018','18',"But the Syrians fled before Israel; and David slew of the Syrians seven thousand \emph{men} \emph{which} \emph{fought} \emph{in} chariots, and forty thousand footmen, and killed Shophach the captain of the host."],
    ['1Chronicles','1Chr','19','019','19',"And when the servants of Hadarezer saw that they were put to the worse before Israel, they made peace with David, and became his servants: neither would the Syrians help the children of Ammon any more."]
]

FirstChronicles20 = [
    ['1Chronicles','1Chr','20','001','1',"And it came to pass, that after the year was expired, at the time that kings go out \emph{to} \emph{battle}, Joab led forth the power of the army, and wasted the country of the children of Ammon, and came and besieged Rabbah. But David tarried at Jerusalem. And Joab smote Rabbah, and destroyed it."],
['1Chronicles','1Chr','20','002','2',"And David took the crown of their king from off his head, and found it to weigh a talent of gold, and \emph{there} \emph{were} precious stones in it; and it was set upon David's head: and he brought also exceeding much spoil out of the city."],
['1Chronicles','1Chr','20','003','3',"And he brought out the people that \emph{were} in it, and cut \emph{them} with saws, and with harrows of iron, and with axes. Even so dealt David with all the cities of the children of Ammon. And David and all the people returned to Jerusalem."],
['1Chronicles','1Chr','20','004','4',"And it came to pass after this, that there arose war at Gezer with the Philistines; at which time Sibbechai the Hushathite slew Sippai, \emph{that} \emph{was} of the children of the giant: and they were subdued."],
['1Chronicles','1Chr','20','005','5',"And there was war again with the Philistines; and Elhanan the son of Jair slew Lahmi the brother of Goliath the Gittite, whose spear staff \emph{was} like a weaver's beam."],
['1Chronicles','1Chr','20','006','6',"And yet again there was war at Gath, where was a man of \emph{great} stature, whose fingers and toes \emph{were} four and twenty, six \emph{on} \emph{each} \emph{hand}, and six \emph{on} \emph{each} \emph{foot}: and he also was the son of the giant."],
['1Chronicles','1Chr','20','007','7',"But when he defied Israel, Jonathan the son of Shimea David's brother slew him."],
['1Chronicles','1Chr','20','008','8',"These were born unto the giant in Gath; and they fell by the hand of David, and by the hand of his servants."]
]

FirstChronicles21 = [
    ['1Chronicles','1Chr','21','001','1',"And Satan stood up against Israel, and provoked David to number Israel."],
['1Chronicles','1Chr','21','002','2',"And David said to Joab and to the rulers of the people, Go, number Israel from Beer-sheba even to Dan; and bring the number of them to me, that I may know \emph{it}."],
['1Chronicles','1Chr','21','003','3',"And Joab answered, The LORD make his people an hundred times so many more as they \emph{be}: but, my lord the king, \emph{are} they not all my lord's servants? why then doth my lord require this thing? why will he be a cause of trespass to Israel?"],
['1Chronicles','1Chr','21','004','4',"Nevertheless the king's word prevailed against Joab. Wherefore Joab departed, and went throughout all Israel, and came to Jerusalem."],
['1Chronicles','1Chr','21','005','5',"And Joab gave the sum of the number of the people unto David. And all \emph{they} \emph{of} Israel were a thousand thousand and an hundred thousand men that drew sword: and Judah \emph{was} four hundred threescore and ten thousand men that drew sword."],
['1Chronicles','1Chr','21','006','6',"But Levi and Benjamin counted he not among them: for the king's word was abominable to Joab."],
['1Chronicles','1Chr','21','007','7',"And God was displeased with this thing; therefore he smote Israel."],
['1Chronicles','1Chr','21','008','8',"And David said unto God, I have sinned greatly, because I have done this thing: but now, I beseech thee, do away the iniquity of thy servant; for I have done very foolishly."],
['1Chronicles','1Chr','21','009','9',"And the LORD spake unto Gad, David's seer, saying,"],
['1Chronicles','1Chr','21','010','10',"Go and tell David, saying, Thus saith the LORD, I offer thee three \emph{things}: choose thee one of them, that I may do \emph{it} unto thee."],
['1Chronicles','1Chr','21','011','11',"So Gad came to David, and said unto him, Thus saith the LORD, Choose thee"],
['1Chronicles','1Chr','21','012','12',"Either three years' famine; or three months to be destroyed before thy foes, while that the sword of thine enemies overtaketh \emph{thee}; or else three days the sword of the LORD, even the pestilence, in the land, and the angel of the LORD destroying throughout all the coasts of Israel. Now therefore advise thyself what word I shall bring again to him that sent me."],
['1Chronicles','1Chr','21','013','13',"And David said unto Gad, I am in a great strait: let me fall now into the hand of the LORD; for very great \emph{are} his mercies: but let me not fall into the hand of man."],
['1Chronicles','1Chr','21','014','14',"So the LORD sent pestilence upon Israel: and there fell of Israel seventy thousand men."],
['1Chronicles','1Chr','21','015','15',"And God sent an angel unto Jerusalem to destroy it: and as he was destroying, the LORD beheld, and he repented him of the evil, and said to the angel that destroyed, It is enough, stay now thine hand. And the angel of the LORD stood by the threshingfloor of Ornan the Jebusite."],
['1Chronicles','1Chr','21','016','16',"And David lifted up his eyes, and saw the angel of the LORD stand between the earth and the heaven, having a drawn sword in his hand stretched out over Jerusalem. Then David and the elders \emph{of} \emph{Israel,} \emph{who} \emph{were} clothed in sackcloth, fell upon their faces."],
['1Chronicles','1Chr','21','017','17',"And David said unto God, \emph{Is} \emph{it} not I \emph{that} commanded the people to be numbered? even I it is that have sinned and done evil indeed; but \emph{as} \emph{for} these sheep, what have they done? let thine hand, I pray thee, O LORD my God, be on me, and on my father's house; but not on thy people, that they should be plagued."],
['1Chronicles','1Chr','21','018','18',"Then the angel of the LORD commanded Gad to say to David, that David should go up, and set up an altar unto the LORD in the threshingfloor of Ornan the Jebusite."],
['1Chronicles','1Chr','21','019','19',"And David went up at the saying of Gad, which he spake in the name of the LORD."],
['1Chronicles','1Chr','21','020','20',"And Ornan turned back, and saw the angel; and his four sons with him hid themselves. Now Ornan was threshing wheat."],
['1Chronicles','1Chr','21','021','21',"And as David came to Ornan, Ornan looked and saw David, and went out of the threshingfloor, and bowed himself to David with \emph{his} face to the ground."],
['1Chronicles','1Chr','21','022','22',"Then David said to Ornan, Grant me the place of \emph{this} threshingfloor, that I may build an altar therein unto the LORD: thou shalt grant it me for the full price: that the plague may be stayed from the people."],
['1Chronicles','1Chr','21','023','23',"And Ornan said unto David, Take \emph{it} to thee, and let my lord the king do \emph{that} \emph{which} \emph{is} good in his eyes: lo, I give \emph{thee} the oxen \emph{also} for burnt offerings, and the threshing instruments for wood, and the wheat for the meat offering; I give it all."],
['1Chronicles','1Chr','21','024','24',"And king David said to Ornan, Nay; but I will verily buy it for the full price: for I will not take \emph{that} which \emph{is} thine for the LORD, nor offer burnt offerings without cost."],
['1Chronicles','1Chr','21','025','25',"So David gave to Ornan for the place six hundred shekels of gold by weight."],
['1Chronicles','1Chr','21','026','26',"And David built there an altar unto the LORD, and offered burnt offerings and peace offerings, and called upon the LORD; and he answered him from heaven by fire upon the altar of burnt offering."],
['1Chronicles','1Chr','21','027','27',"And the LORD commanded the angel; and he put up his sword again into the sheath thereof."],
['1Chronicles','1Chr','21','028','28',"At that time when David saw that the LORD had answered him in the threshingfloor of Ornan the Jebusite, then he sacrificed there."],
['1Chronicles','1Chr','21','029','29',"For the tabernacle of the LORD, which Moses made in the wilderness, and the altar of the burnt offering, \emph{were} at that season in the high place at Gibeon."],
['1Chronicles','1Chr','21','030','30',"But David could not go before it to enquire of God: for he was afraid because of the sword of the angel of the LORD."]
]

FirstChronicles22 = [
    ['1Chronicles','1Chr','22','001','1',"Then David said, This \emph{is} the house of the LORD God, and this \emph{is} the altar of the burnt offering for Israel."],
['1Chronicles','1Chr','22','002','2',"And David commanded to gather together the strangers that \emph{were} in the land of Israel; and he set masons to hew wrought stones to build the house of God."],
['1Chronicles','1Chr','22','003','3',"And David prepared iron in abundance for the nails for the doors of the gates, and for the joinings; and brass in abundance without weight;"],
['1Chronicles','1Chr','22','004','4',"Also cedar trees in abundance: for the Zidonians and they of Tyre brought much cedar wood to David."],
['1Chronicles','1Chr','22','005','5',"And David said, Solomon my son \emph{is} young and tender, and the house \emph{that} \emph{is} to be builded for the LORD \emph{must} \emph{be} exceeding magnifical, of fame and of glory throughout all countries: I will \emph{therefore} now make preparation for it. So David prepared abundantly before his death."],
['1Chronicles','1Chr','22','006','6',"Then he called for Solomon his son, and charged him to build an house for the LORD God of Israel."],
['1Chronicles','1Chr','22','007','7',"And David said to Solomon, My son, as for me, it was in my mind to build an house unto the name of the LORD my God:"],
['1Chronicles','1Chr','22','008','8',"But the word of the LORD came to me, saying, Thou hast shed blood abundantly, and hast made great wars: thou shalt not build an house unto my name, because thou hast shed much blood upon the earth in my sight."],
['1Chronicles','1Chr','22','009','9',"Behold, a son shall be born to thee, who shall be a man of rest; and I will give him rest from all his enemies round about: for his name shall be Solomon, and I will give peace and quietness unto Israel in his days."],
['1Chronicles','1Chr','22','010','10',"He shall build an house for my name; and he shall be my son, and I \emph{will} \emph{be} his father; and I will establish the throne of his kingdom over Israel for ever."],
['1Chronicles','1Chr','22','011','11',"Now, my son, the LORD be with thee; and prosper thou, and build the house of the LORD thy God, as he hath said of thee."],
['1Chronicles','1Chr','22','012','12',"Only the LORD give thee wisdom and understanding, and give thee charge concerning Israel, that thou mayest keep the law of the LORD thy God."],
['1Chronicles','1Chr','22','013','13',"Then shalt thou prosper, if thou takest heed to fulfil the statutes and judgments which the LORD charged Moses with concerning Israel: be strong, and of good courage; dread not, nor be dismayed."],
['1Chronicles','1Chr','22','014','14',"Now, behold, in my trouble I have prepared for the house of the LORD an hundred thousand talents of gold, and a thousand thousand talents of silver; and of brass and iron without weight; for it is in abundance: timber also and stone have I prepared; and thou mayest add thereto."],
['1Chronicles','1Chr','22','015','15',"Moreover \emph{there} \emph{are} workmen with thee in abundance, hewers and workers of stone and timber, and all manner of cunning men for every manner of work."],
['1Chronicles','1Chr','22','016','16',"Of the gold, the silver, and the brass, and the iron, \emph{there} \emph{is} no number. Arise \emph{therefore}, and be doing, and the LORD be with thee."],
['1Chronicles','1Chr','22','017','17',"David also commanded all the princes of Israel to help Solomon his son, \emph{saying},"],
['1Chronicles','1Chr','22','018','18',"\emph{Is} not the LORD your God with you? and hath he \emph{not} given you rest on every side? for he hath given the inhabitants of the land into mine hand; and the land is subdued before the LORD, and before his people."],
['1Chronicles','1Chr','22','019','19',"Now set your heart and your soul to seek the LORD your God; arise therefore, and build ye the sanctuary of the LORD God, to bring the ark of the covenant of the LORD, and the holy vessels of God, into the house that is to be built to the name of the LORD."]
]

FirstChronicles23 = [
    ['1Chronicles','1Chr','23','001','1',"So when David was old and full of days, he made Solomon his son king over Israel."],
['1Chronicles','1Chr','23','002','2',"And he gathered together all the princes of Israel, with the priests and the Levites."],
['1Chronicles','1Chr','23','003','3',"Now the Levites were numbered from the age of thirty years and upward: and their number by their polls, man by man, was thirty and eight thousand."],
['1Chronicles','1Chr','23','004','4',"Of which, twenty and four thousand \emph{were} to set forward the work of the house of the LORD; and six thousand \emph{were} officers and judges:"],
['1Chronicles','1Chr','23','005','5',"Moreover four thousand \emph{were} porters; and four thousand praised the LORD with the instruments which I made, \emph{said} \emph{David}, to praise \emph{therewith}."],
['1Chronicles','1Chr','23','006','6',"And David divided them into courses among the sons of Levi, \emph{namely}, Gershon, Kohath, and Merari."],
['1Chronicles','1Chr','23','007','7',"Of the Gershonites \emph{were}, Laadan, and Shimei."],
['1Chronicles','1Chr','23','008','8',"The sons of Laadan; the chief \emph{was} Jehiel, and Zetham, and Joel, three."],
['1Chronicles','1Chr','23','009','9',"The sons of Shimei; Shelomith, and Haziel, and Haran, three. These \emph{were} the chief of the fathers of Laadan."],
['1Chronicles','1Chr','23','010','10',"And the sons of Shimei \emph{were}, Jahath, Zina, and Jeush, and Beriah. These four \emph{were} the sons of Shimei."],
['1Chronicles','1Chr','23','011','11',"And Jahath was the chief, and Zizah the second: but Jeush and Beriah had not many sons; therefore they were in one reckoning, according to \emph{their} father's house."],
['1Chronicles','1Chr','23','012','12',"The sons of Kohath; Amram, Izhar, Hebron, and Uzziel, four."],
['1Chronicles','1Chr','23','013','13',"The sons of Amram; Aaron and Moses: and Aaron was separated, that he should sanctify the most holy things, he and his sons for ever, to burn incense before the LORD, to minister unto him, and to bless in his name for ever."],
['1Chronicles','1Chr','23','014','14',"Now \emph{concerning} Moses the man of God, his sons were named of the tribe of Levi."],
['1Chronicles','1Chr','23','015','15',"The sons of Moses \emph{were}, Gershom, and Eliezer."],
['1Chronicles','1Chr','23','016','16',"Of the sons of Gershom, Shebuel \emph{was} the chief."],
['1Chronicles','1Chr','23','017','17',"And the sons of Eliezer \emph{were}, Rehabiah the chief. And Eliezer had none other sons; but the sons of Rehabiah were very many."],
['1Chronicles','1Chr','23','018','18',"Of the sons of Izhar; Shelomith the chief."],
['1Chronicles','1Chr','23','019','19',"Of the sons of Hebron; Jeriah the first, Amariah the second, Jahaziel the third, and Jekameam the fourth."],
['1Chronicles','1Chr','23','020','20',"Of the sons of Uzziel; Michah the first, and Jesiah the second."],
['1Chronicles','1Chr','23','021','21',"The sons of Merari; Mahli, and Mushi. The sons of Mahli; Eleazar, and Kish."],
['1Chronicles','1Chr','23','022','22',"And Eleazar died, and had no sons, but daughters: and their brethren the sons of Kish took them."],
['1Chronicles','1Chr','23','023','23',"The sons of Mushi; Mahli, and Eder, and Jeremoth, three."],
['1Chronicles','1Chr','23','024','24',"These \emph{were} the sons of Levi after the house of their fathers; \emph{even} the chief of the fathers, as they were counted by number of names by their polls, that did the work for the service of the house of the LORD, from the age of twenty years and upward."],
['1Chronicles','1Chr','23','025','25',"For David said, The LORD God of Israel hath given rest unto his people, that they may dwell in Jerusalem for ever:"],
['1Chronicles','1Chr','23','026','26',"And also unto the Levites; they shall no \emph{more} carry the tabernacle, nor any vessels of it for the service thereof."],
['1Chronicles','1Chr','23','027','27',"For by the last words of David the Levites \emph{were} numbered from twenty years old and above:"],
['1Chronicles','1Chr','23','028','28',"Because their office \emph{was} to wait on the sons of Aaron for the service of the house of the LORD, in the courts, and in the chambers, and in the purifying of all holy things, and the work of the service of the house of God;"],
['1Chronicles','1Chr','23','029','29',"Both for the shewbread, and for the fine flour for meat offering, and for the unleavened cakes, and for \emph{that} \emph{which} \emph{is} \emph{baked} \emph{in} the pan, and for that which is fried, and for all manner of measure and size;"],
['1Chronicles','1Chr','23','030','30',"And to stand every morning to thank and praise the LORD, and likewise at even;"],
['1Chronicles','1Chr','23','031','31',"And to offer all burnt sacrifices unto the LORD in the sabbaths, in the new moons, and on the set feasts, by number, according to the order commanded unto them, continually before the LORD:"],
['1Chronicles','1Chr','23','032','32',"And that they should keep the charge of the tabernacle of the congregation, and the charge of the holy \emph{place}, and the charge of the sons of Aaron their brethren, in the service of the house of the LORD."]
]

FirstChronicles24 = [
    ['1Chronicles','1Chr','24','001','1',"Now \emph{these} \emph{are} the divisions of the sons of Aaron. The sons of Aaron; Nadab, and Abihu, Eleazar, and Ithamar."],
['1Chronicles','1Chr','24','002','2',"But Nadab and Abihu died before their father, and had no children: therefore Eleazar and Ithamar executed the priest's office."],
['1Chronicles','1Chr','24','003','3',"And David distributed them, both Zadok of the sons of Eleazar, and Ahimelech of the sons of Ithamar, according to their offices in their service."],
['1Chronicles','1Chr','24','004','4',"And there were more chief men found of the sons of Eleazar than of the sons of Ithamar; and \emph{thus} were they divided. Among the sons of Eleazar \emph{there} \emph{were} sixteen chief men of the house of \emph{their} fathers, and eight among the sons of Ithamar according to the house of their fathers."],
['1Chronicles','1Chr','24','005','5',"Thus were they divided by lot, one sort with another; for the governors of the sanctuary, and governors \emph{of} \emph{the} \emph{house} of God, were of the sons of Eleazar, and of the sons of Ithamar."],
['1Chronicles','1Chr','24','006','6',"And Shemaiah the son of Nethaneel the scribe, \emph{one} of the Levites, wrote them before the king, and the princes, and Zadok the priest, and Ahimelech the son of Abiathar, and \emph{before} the chief of the fathers of the priests and Levites: one principal household being taken for Eleazar, and \emph{one} taken for Ithamar."],
['1Chronicles','1Chr','24','007','7',"Now the first lot came forth to Jehoiarib, the second to Jedaiah,"],
['1Chronicles','1Chr','24','008','8',"The third to Harim, the fourth to Seorim,"],
['1Chronicles','1Chr','24','009','9',"The fifth to Malchijah, the sixth to Mijamin,"],
['1Chronicles','1Chr','24','010','10',"The seventh to Hakkoz, the eighth to Abijah,"],
['1Chronicles','1Chr','24','011','11',"The ninth to Jeshua, the tenth to Shecaniah,"],
['1Chronicles','1Chr','24','012','12',"The eleventh to Eliashib, the twelfth to Jakim,"],
['1Chronicles','1Chr','24','013','13',"The thirteenth to Huppah, the fourteenth to Jeshebeab,"],
['1Chronicles','1Chr','24','014','14',"The fifteenth to Bilgah, the sixteenth to Immer,"],
['1Chronicles','1Chr','24','015','15',"The seventeenth to Hezir, the eighteenth to Aphses,"],
['1Chronicles','1Chr','24','016','16',"The nineteenth to Pethahiah, the twentieth to Jehezekel,"],
['1Chronicles','1Chr','24','017','17',"The one and twentieth to Jachin, the two and twentieth to Gamul,"],
['1Chronicles','1Chr','24','018','18',"The three and twentieth to Delaiah, the four and twentieth to Maaziah."],
['1Chronicles','1Chr','24','019','19',"These \emph{were} the orderings of them in their service to come into the house of the LORD, according to their manner, under Aaron their father, as the LORD God of Israel had commanded him."],
['1Chronicles','1Chr','24','020','20',"And the rest of the sons of Levi \emph{were} \emph{these}: Of the sons of Amram; Shubael: of the sons of Shubael; Jehdeiah."],
['1Chronicles','1Chr','24','021','21',"Concerning Rehabiah: of the sons of Rehabiah, the first \emph{was} Isshiah."],
['1Chronicles','1Chr','24','022','22',"Of the Izharites; Shelomoth: of the sons of Shelomoth; Jahath."],
['1Chronicles','1Chr','24','023','23',"And the sons \emph{of} \emph{Hebron}; Jeriah \emph{the} \emph{first}, Amariah the second, Jahaziel the third, Jekameam the fourth."],
['1Chronicles','1Chr','24','024','24',"\emph{Of} the sons of Uzziel; Michah: of the sons of Michah; Shamir."],
['1Chronicles','1Chr','24','025','25',"The brother of Michah \emph{was} Isshiah: of the sons of Isshiah; Zechariah."],
['1Chronicles','1Chr','24','026','26',"The sons of Merari \emph{were} Mahli and Mushi: the sons of Jaaziah; Beno."],
['1Chronicles','1Chr','24','027','27',"The sons of Merari by Jaaziah; Beno, and Shoham, and Zaccur, and Ibri."],
['1Chronicles','1Chr','24','028','28',"Of Mahli \emph{came} Eleazar, who had no sons."],
['1Chronicles','1Chr','24','029','29',"Concerning Kish: the son of Kish \emph{was} Jerahmeel."],
['1Chronicles','1Chr','24','030','30',"The sons also of Mushi; Mahli, and Eder, and Jerimoth. These \emph{were} the sons of the Levites after the house of their fathers."],
['1Chronicles','1Chr','24','031','31',"These likewise cast lots over against their brethren the sons of Aaron in the presence of David the king, and Zadok, and Ahimelech, and the chief of the fathers of the priests and Levites, even the principal fathers over against their younger brethren."]
]

FirstChronicles25 = [
    ['1Chronicles','1Chr','25','001','1',"Moreover David and the captains of the host separated to the service of the sons of Asaph, and of Heman, and of Jeduthun, who should prophesy with harps, with psalteries, and with cymbals: and the number of the workmen according to their service was:"],
['1Chronicles','1Chr','25','002','2',"Of the sons of Asaph; Zaccur, and Joseph, and Nethaniah, and Asarelah, the sons of Asaph under the hands of Asaph, which prophesied according to the order of the king."],
['1Chronicles','1Chr','25','003','3',"Of Jeduthun: the sons of Jeduthun; Gedaliah, and Zeri, and Jeshaiah, Hashabiah, and Mattithiah, six, under the hands of their father Jeduthun, who prophesied with a harp, to give thanks and to praise the LORD."],
['1Chronicles','1Chr','25','004','4',"Of Heman: the sons of Heman; Bukkiah, Mattaniah, Uzziel, Shebuel, and Jerimoth, Hananiah, Hanani, Eliathah, Giddalti, and Romamti-ezer, Joshbekashah, Mallothi, Hothir, \emph{and} Mahazioth:"],
['1Chronicles','1Chr','25','005','5',"All these \emph{were} the sons of Heman the king's seer in the words of God, to lift up the horn. And God gave to Heman fourteen sons and three daughters."],
['1Chronicles','1Chr','25','006','6',"All these \emph{were} under the hands of their father for song \emph{in} the house of the LORD, with cymbals, psalteries, and harps, for the service of the house of God, according to the king's order to Asaph, Jeduthun, and Heman."],
['1Chronicles','1Chr','25','007','7',"So the number of them, with their brethren that were instructed in the songs of the LORD, \emph{even} all that were cunning, was two hundred fourscore and eight."],
['1Chronicles','1Chr','25','008','8',"And they cast lots, ward against \emph{ward}, as well the small as the great, the teacher as the scholar."],
['1Chronicles','1Chr','25','009','9',"Now the first lot came forth for Asaph to Joseph: the second to Gedaliah, who with his brethren and sons \emph{were} twelve:"],
['1Chronicles','1Chr','25','010','10',"The third to Zaccur, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','011','11',"The fourth to Izri, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','012','12',"The fifth to Nethaniah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','013','13',"The sixth to Bukkiah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','014','14',"The seventh to Jesharelah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','015','15',"The eighth to Jeshaiah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','016','16',"The ninth to Mattaniah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','017','17',"The tenth to Shimei, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','018','18',"The eleventh to Azareel, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','019','19',"The twelfth to Hashabiah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','020','20',"The thirteenth to Shubael, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','021','21',"The fourteenth to Mattithiah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','022','22',"The fifteenth to Jeremoth, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','023','23',"The sixteenth to Hananiah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','024','24',"The seventeenth to Joshbekashah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','025','25',"The eighteenth to Hanani, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','026','26',"The nineteenth to Mallothi, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','027','27',"The twentieth to Eliathah, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','028','28',"The one and twentieth to Hothir, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','029','29',"The two and twentieth to Giddalti, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','030','30',"The three and twentieth to Mahazioth, \emph{he}, his sons, and his brethren, \emph{were} twelve:"],
['1Chronicles','1Chr','25','031','31',"The four and twentieth to Romamti-ezer, \emph{he}, his sons, and his brethren, \emph{were} twelve."]
]

FirstChronicles26 = [
    ['1Chronicles','1Chr','26','001','1',"Concerning the divisions of the porters: Of the Korhites \emph{was} Meshelemiah the son of Kore, of the sons of Asaph."],
['1Chronicles','1Chr','26','002','2',"And the sons of Meshelemiah \emph{were}, Zechariah the firstborn, Jediael the second, Zebadiah the third, Jathniel the fourth,"],
['1Chronicles','1Chr','26','003','3',"Elam the fifth, Jehohanan the sixth, Elioenai the seventh."],
['1Chronicles','1Chr','26','004','4',"Moreover the sons of Obed-edom \emph{were}, Shemaiah the firstborn, Jehozabad the second, Joah the third, and Sacar the fourth, and Nethaneel the fifth,"],
['1Chronicles','1Chr','26','005','5',"Ammiel the sixth, Issachar the seventh, Peulthai the eighth: for God blessed him."],
['1Chronicles','1Chr','26','006','6',"Also unto Shemaiah his son were sons born, that ruled throughout the house of their father: for they \emph{were} mighty men of valour."],
['1Chronicles','1Chr','26','007','7',"The sons of Shemaiah; Othni, and Rephael, and Obed, Elzabad, whose brethren \emph{were} strong men, Elihu, and Semachiah."],
['1Chronicles','1Chr','26','008','8',"All these of the sons of Obed-edom: they and their sons and their brethren, able men for strength for the service, \emph{were} threescore and two of Obed-edom."],
['1Chronicles','1Chr','26','009','9',"And Meshelemiah had sons and brethren, strong men, eighteen."],
['1Chronicles','1Chr','26','010','10',"Also Hosah, of the children of Merari, had sons; Simri the chief, (for \emph{though} he was not the firstborn, yet his father made him the chief;)"],
['1Chronicles','1Chr','26','011','11',"Hilkiah the second, Tebaliah the third, Zechariah the fourth: all the sons and brethren of Hosah \emph{were} thirteen."],
['1Chronicles','1Chr','26','012','12',"Among these \emph{were} the divisions of the porters, \emph{even} among the chief men, \emph{having} wards one against another, to minister in the house of the LORD."],
['1Chronicles','1Chr','26','013','13',"And they cast lots, as well the small as the great, according to the house of their fathers, for every gate."],
['1Chronicles','1Chr','26','014','14',"And the lot eastward fell to Shelemiah. Then for Zechariah his son, a wise counsellor, they cast lots; and his lot came out northward."],
['1Chronicles','1Chr','26','015','15',"To Obed-edom southward; and to his sons the house of Asuppim."],
['1Chronicles','1Chr','26','016','16',"To Shuppim and Hosah \emph{the} \emph{lot} \emph{came} \emph{forth} westward, with the gate Shallecheth, by the causeway of the going up, ward against ward."],
['1Chronicles','1Chr','26','017','17',"Eastward \emph{were} six Levites, northward four a day, southward four a day, and toward Asuppim two \emph{and} two."],
['1Chronicles','1Chr','26','018','18',"At Parbar westward, four at the causeway, \emph{and} two at Parbar."],
['1Chronicles','1Chr','26','019','19',"These \emph{are} the divisions of the porters among the sons of Kore, and among the sons of Merari."],
['1Chronicles','1Chr','26','020','20',"And of the Levites, Ahijah \emph{was} over the treasures of the house of God, and over the treasures of the dedicated things."],
['1Chronicles','1Chr','26','021','21',"\emph{As} \emph{concerning} the sons of Laadan; the sons of the Gershonite Laadan, chief fathers, \emph{even} of Laadan the Gershonite, \emph{were} Jehieli."],
['1Chronicles','1Chr','26','022','22',"The sons of Jehieli; Zetham, and Joel his brother, \emph{which} \emph{were} over the treasures of the house of the LORD."],
['1Chronicles','1Chr','26','023','23',"Of the Amramites, \emph{and} the Izharites, the Hebronites, \emph{and} the Uzzielites:"],
['1Chronicles','1Chr','26','024','24',"And Shebuel the son of Gershom, the son of Moses, \emph{was} ruler of the treasures."],
['1Chronicles','1Chr','26','025','25',"And his brethren by Eliezer; Rehabiah his son, and Jeshaiah his son, and Joram his son, and Zichri his son, and Shelomith his son."],
['1Chronicles','1Chr','26','026','26',"Which Shelomith and his brethren \emph{were} over all the treasures of the dedicated things, which David the king, and the chief fathers, the captains over thousands and hundreds, and the captains of the host, had dedicated."],
['1Chronicles','1Chr','26','027','27',"Out of the spoils won in battles did they dedicate to maintain the house of the LORD."],
['1Chronicles','1Chr','26','028','28',"And all that Samuel the seer, and Saul the son of Kish, and Abner the son of Ner, and Joab the son of Zeruiah, had dedicated; \emph{and} whosoever had dedicated \emph{any} \emph{thing,} \emph{it} \emph{was} under the hand of Shelomith, and of his brethren."],
['1Chronicles','1Chr','26','029','29',"Of the Izharites, Chenaniah and his sons \emph{were} for the outward business over Israel, for officers and judges."],
['1Chronicles','1Chr','26','030','30',"\emph{And} of the Hebronites, Hashabiah and his brethren, men of valour, a thousand and seven hundred, \emph{were} officers among them of Israel on this side Jordan westward in all the business of the LORD, and in the service of the king."],
['1Chronicles','1Chr','26','031','31',"Among the Hebronites \emph{was} Jerijah the chief, \emph{even} among the Hebronites, according to the generations of his fathers. In the fortieth year of the reign of David they were sought for, and there were found among them mighty men of valour at Jazer of Gilead."],
['1Chronicles','1Chr','26','032','32',"And his brethren, men of valour, \emph{were} two thousand and seven hundred chief fathers, whom king David made rulers over the Reubenites, the Gadites, and the half tribe of Manasseh, for every matter pertaining to God, and affairs of the king."]
]

FirstChronicles27 = [
    ['1Chronicles','1Chr','27','001','1',"Now the children of Israel after their number, \emph{to} \emph{wit}, the chief fathers and captains of thousands and hundreds, and their officers that served the king in any matter of the courses, which came in and went out month by month throughout all the months of the year, of every course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','002','2',"Over the first course for the first month \emph{was} Jashobeam the son of Zabdiel: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','003','3',"Of the children of Perez \emph{was} the chief of all the captains of the host for the first month."],
['1Chronicles','1Chr','27','004','4',"And over the course of the second month \emph{was} Dodai an Ahohite, and of his course \emph{was} Mikloth also the ruler: in his course likewise \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','005','5',"The third captain of the host for the third month \emph{was} Benaiah the son of Jehoiada, a chief priest: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','006','6',"This \emph{is} \emph{that} Benaiah, \emph{who} \emph{was} mighty \emph{among} the thirty, and above the thirty: and in his course \emph{was} Ammizabad his son."],
['1Chronicles','1Chr','27','007','7',"The fourth \emph{captain} for the fourth month \emph{was} Asahel the brother of Joab, and Zebadiah his son after him: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','008','8',"The fifth captain for the fifth month \emph{was} Shamhuth the Izrahite: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','009','9',"The sixth \emph{captain} for the sixth month \emph{was} Ira the son of Ikkesh the Tekoite: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','010','10',"The seventh \emph{captain} for the seventh month \emph{was} Helez the Pelonite, of the children of Ephraim: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','011','11',"The eighth \emph{captain} for the eighth month \emph{was} Sibbecai the Hushathite, of the Zarhites: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','012','12',"The ninth \emph{captain} for the ninth month \emph{was} Abiezer the Anetothite, of the Benjamites: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','013','13',"The tenth \emph{captain} for the tenth month \emph{was} Maharai the Netophathite, of the Zarhites: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','014','14',"The eleventh \emph{captain} for the eleventh month \emph{was} Benaiah the Pirathonite, of the children of Ephraim: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','015','15',"The twelfth \emph{captain} for the twelfth month \emph{was} Heldai the Netophathite, of Othniel: and in his course \emph{were} twenty and four thousand."],
['1Chronicles','1Chr','27','016','16',"Furthermore over the tribes of Israel: the ruler of the Reubenites \emph{was} Eliezer the son of Zichri: of the Simeonites, Shephatiah the son of Maachah:"],
['1Chronicles','1Chr','27','017','17',"Of the Levites, Hashabiah the son of Kemuel: of the Aaronites, Zadok:"],
['1Chronicles','1Chr','27','018','18',"Of Judah, Elihu, \emph{one} of the brethren of David: of Issachar, Omri the son of Michael:"],
['1Chronicles','1Chr','27','019','19',"Of Zebulun, Ishmaiah the son of Obadiah: of Naphtali, Jerimoth the son of Azriel:"],
['1Chronicles','1Chr','27','020','20',"Of the children of Ephraim, Hoshea the son of Azaziah: of the half tribe of Manasseh, Joel the son of Pedaiah:"],
['1Chronicles','1Chr','27','021','21',"Of the half \emph{tribe} of Manasseh in Gilead, Iddo the son of Zechariah: of Benjamin, Jaasiel the son of Abner:"],
['1Chronicles','1Chr','27','022','22',"Of Dan, Azareel the son of Jeroham. These \emph{were} the princes of the tribes of Israel."],
['1Chronicles','1Chr','27','023','23',"But David took not the number of them from twenty years old and under: because the LORD had said he would increase Israel like to the stars of the heavens."],
['1Chronicles','1Chr','27','024','24',"Joab the son of Zeruiah began to number, but he finished not, because there fell wrath for it against Israel; neither was the number put in the account of the chronicles of king David."],
['1Chronicles','1Chr','27','025','25',"And over the king's treasures \emph{was} Azmaveth the son of Adiel: and over the storehouses in the fields, in the cities, and in the villages, and in the castles, \emph{was} Jehonathan the son of Uzziah:"],
['1Chronicles','1Chr','27','026','26',"And over them that did the work of the field for tillage of the ground \emph{was} Ezri the son of Chelub:"],
['1Chronicles','1Chr','27','027','27',"And over the vineyards \emph{was} Shimei the Ramathite: over the increase of the vineyards for the wine cellars \emph{was} Zabdi the Shiphmite:"],
['1Chronicles','1Chr','27','028','28',"And over the olive trees and the sycomore trees that \emph{were} in the low plains \emph{was} Baal-hanan the Gederite: and over the cellars of oil \emph{was} Joash:"],
['1Chronicles','1Chr','27','029','29',"And over the herds that fed in Sharon \emph{was} Shitrai the Sharonite: and over the herds \emph{that} \emph{were} in the valleys \emph{was} Shaphat the son of Adlai:"],
['1Chronicles','1Chr','27','030','30',"Over the camels also \emph{was} Obil the Ishmaelite: and over the asses \emph{was} Jehdeiah the Meronothite:"],
['1Chronicles','1Chr','27','031','31',"And over the flocks \emph{was} Jaziz the Hagerite. All these \emph{were} the rulers of the substance which \emph{was} king David's."],
['1Chronicles','1Chr','27','032','32',"Also Jonathan David's uncle was a counsellor, a wise man, and a scribe: and Jehiel the son of Hachmoni \emph{was} with the king's sons:"],
['1Chronicles','1Chr','27','033','33',"And Ahithophel \emph{was} the king's counsellor: and Hushai the Archite \emph{was} the king's companion:"],
['1Chronicles','1Chr','27','034','34',"And after Ahithophel \emph{was} Jehoiada the son of Benaiah, and Abiathar: and the general of the king's army \emph{was} Joab."]
]

FirstChronicles28 = [
    ['1Chronicles','1Chr','28','001','1',"And David assembled all the princes of Israel, the princes of the tribes, and the captains of the companies that ministered to the king by course, and the captains over the thousands, and captains over the hundreds, and the stewards over all the substance and possession of the king, and of his sons, with the officers, and with the mighty men, and with all the valiant men, unto Jerusalem."],
['1Chronicles','1Chr','28','002','2',"Then David the king stood up upon his feet, and said, Hear me, my brethren, and my people: \emph{As} \emph{for} \emph{me}, I \emph{had} in mine heart to build an house of rest for the ark of the covenant of the LORD, and for the footstool of our God, and had made ready for the building:"],
['1Chronicles','1Chr','28','003','3',"But God said unto me, Thou shalt not build an house for my name, because thou \emph{hast} \emph{been} a man of war, and hast shed blood."],
['1Chronicles','1Chr','28','004','4',"Howbeit the LORD God of Israel chose me before all the house of my father to be king over Israel for ever: for he hath chosen Judah \emph{to} \emph{be} the ruler; and of the house of Judah, the house of my father; and among the sons of my father he liked me to make \emph{me} king over all Israel:"],
['1Chronicles','1Chr','28','005','5',"And of all my sons, (for the LORD hath given me many sons,) he hath chosen Solomon my son to sit upon the throne of the kingdom of the LORD over Israel."],
['1Chronicles','1Chr','28','006','6',"And he said unto me, Solomon thy son, he shall build my house and my courts: for I have chosen him \emph{to} \emph{be} my son, and I will be his father."],
['1Chronicles','1Chr','28','007','7',"Moreover I will establish his kingdom for ever, if he be constant to do my commandments and my judgments, as at this day."],
['1Chronicles','1Chr','28','008','8',"Now therefore in the sight of all Israel the congregation of the LORD, and in the audience of our God, keep and seek for all the commandments of the LORD your God: that ye may possess this good land, and leave \emph{it} for an inheritance for your children after you for ever."],
['1Chronicles','1Chr','28','009','9',"And thou, Solomon my son, know thou the God of thy father, and serve him with a perfect heart and with a willing mind: for the LORD searcheth all hearts, and understandeth all the imaginations of the thoughts: if thou seek him, he will be found of thee; but if thou forsake him, he will cast thee off for ever."],
['1Chronicles','1Chr','28','010','10',"Take heed now; for the LORD hath chosen thee to build an house for the sanctuary: be strong, and do \emph{it}."],
['1Chronicles','1Chr','28','011','11',"Then David gave to Solomon his son the pattern of the porch, and of the houses thereof, and of the treasuries thereof, and of the upper chambers thereof, and of the inner parlours thereof, and of the place of the mercy seat,"],
['1Chronicles','1Chr','28','012','12',"And the pattern of all that he had by the spirit, of the courts of the house of the LORD, and of all the chambers round about, of the treasuries of the house of God, and of the treasuries of the dedicated things:"],
['1Chronicles','1Chr','28','013','13',"Also for the courses of the priests and the Levites, and for all the work of the service of the house of the LORD, and for all the vessels of service in the house of the LORD."],
['1Chronicles','1Chr','28','014','14',"\emph{He} \emph{gave} of gold by weight for \emph{things} of gold, for all instruments of all manner of service; \emph{silver} \emph{also} for all instruments of silver by weight, for all instruments of every kind of service:"],
['1Chronicles','1Chr','28','015','15',"Even the weight for the candlesticks of gold, and for their lamps of gold, by weight for every candlestick, and for the lamps thereof: and for the candlesticks of silver by weight, \emph{both} for the candlestick, and \emph{also} for the lamps thereof, according to the use of every candlestick."],
['1Chronicles','1Chr','28','016','16',"And by weight \emph{he} \emph{gave} gold for the tables of shewbread, for every table; and \emph{likewise} silver for the tables of silver:"],
['1Chronicles','1Chr','28','017','17',"Also pure gold for the fleshhooks, and the bowls, and the cups: and for the golden basons \emph{he} \emph{gave} \emph{gold} by weight for every bason; and \emph{likewise} \emph{silver} by weight for every bason of silver:"],
['1Chronicles','1Chr','28','018','18',"And for the altar of incense refined gold by weight; and gold for the pattern of the chariot of the cherubims, that spread out \emph{their} \emph{wings}, and covered the ark of the covenant of the LORD."],
['1Chronicles','1Chr','28','019','19',"All \emph{this,} \emph{said} \emph{David}, the LORD made me understand in writing by \emph{his} hand upon me, \emph{even} all the works of this pattern."],
['1Chronicles','1Chr','28','020','20',"And David said to Solomon his son, Be strong and of good courage, and do \emph{it}: fear not, nor be dismayed: for the LORD God, \emph{even} my God, \emph{will} \emph{be} with thee; he will not fail thee, nor forsake thee, until thou hast finished all the work for the service of the house of the LORD."],
['1Chronicles','1Chr','28','021','21',"And, behold, the courses of the priests and the Levites, \emph{even} \emph{they} \emph{shall} \emph{be} \emph{with} \emph{thee} for all the service of the house of God: and \emph{there} \emph{shall} \emph{be} with thee for all manner of workmanship every willing skilful man, for any manner of service: also the princes and all the people \emph{will} \emph{be} wholly at thy commandment."]
]

FirstChronicles29 = [
    ['1Chronicles','1Chr','29','001','1',"Furthermore David the king said unto all the congregation, Solomon my son, whom alone God hath chosen, \emph{is} \emph{yet} young and tender, and the work \emph{is} great: for the palace \emph{is} not for man, but for the LORD God."],
['1Chronicles','1Chr','29','002','2',"Now I have prepared with all my might for the house of my God the gold for \emph{things} \emph{to} \emph{be} \emph{made} of gold, and the silver for \emph{things} of silver, and the brass for \emph{things} of brass, the iron for \emph{things} of iron, and wood for \emph{things} of wood; onyx stones, and \emph{stones} to be set, glistering stones, and of divers colours, and all manner of precious stones, and marble stones in abundance."],
['1Chronicles','1Chr','29','003','3',"Moreover, because I have set my affection to the house of my God, I have of mine own proper good, of gold and silver, \emph{which} I have given to the house of my God, over and above all that I have prepared for the holy house,"],
['1Chronicles','1Chr','29','004','4',"\emph{Even} three thousand talents of gold, of the gold of Ophir, and seven thousand talents of refined silver, to overlay the walls of the houses \emph{withal}:"],
['1Chronicles','1Chr','29','005','5',"The gold for \emph{things} of gold, and the silver for \emph{things} of silver, and for all manner of work \emph{to} \emph{be} \emph{made} by the hands of artificers. And who \emph{then} is willing to consecrate his service this day unto the LORD?"],
['1Chronicles','1Chr','29','006','6',"Then the chief of the fathers and princes of the tribes of Israel, and the captains of thousands and of hundreds, with the rulers of the king's work, offered willingly,"],
['1Chronicles','1Chr','29','007','7',"And gave for the service of the house of God of gold five thousand talents and ten thousand drams, and of silver ten thousand talents, and of brass eighteen thousand talents, and one hundred thousand talents of iron."],
['1Chronicles','1Chr','29','008','8',"And they with whom \emph{precious} stones were found gave \emph{them} to the treasure of the house of the LORD, by the hand of Jehiel the Gershonite."],
['1Chronicles','1Chr','29','009','9',"Then the people rejoiced, for that they offered willingly, because with perfect heart they offered willingly to the LORD: and David the king also rejoiced with great joy."],
['1Chronicles','1Chr','29','010','10',"Wherefore David blessed the LORD before all the congregation: and David said, Blessed \emph{be} thou, LORD God of Israel our father, for ever and ever."],
['1Chronicles','1Chr','29','011','11',"Thine, O LORD, \emph{is} the greatness, and the power, and the glory, and the victory, and the majesty: for all \emph{that} \emph{is} in the heaven and in the earth \emph{is} \emph{thine}; thine \emph{is} the kingdom, O LORD, and thou art exalted as head above all."],
['1Chronicles','1Chr','29','012','12',"Both riches and honour \emph{come} of thee, and thou reignest over all; and in thine hand \emph{is} power and might; and in thine hand \emph{it} \emph{is} to make great, and to give strength unto all."],
['1Chronicles','1Chr','29','013','13',"Now therefore, our God, we thank thee, and praise thy glorious name."],
['1Chronicles','1Chr','29','014','14',"But who \emph{am} I, and what \emph{is} my people, that we should be able to offer so willingly after this sort? for all things \emph{come} of thee, and of thine own have we given thee."],
['1Chronicles','1Chr','29','015','15',"For we \emph{are} strangers before thee, and sojourners, as \emph{were} all our fathers: our days on the earth \emph{are} as a shadow, and \emph{there} \emph{is} none abiding."],
['1Chronicles','1Chr','29','016','16',"O LORD our God, all this store that we have prepared to build thee an house for thine holy name \emph{cometh} of thine hand, and \emph{is} all thine own."],
['1Chronicles','1Chr','29','017','17',"I know also, my God, that thou triest the heart, and hast pleasure in uprightness. As for me, in the uprightness of mine heart I have willingly offered all these things: and now have I seen with joy thy people, which are present here, to offer willingly unto thee."],
['1Chronicles','1Chr','29','018','18',"O LORD God of Abraham, Isaac, and of Israel, our fathers, keep this for ever in the imagination of the thoughts of the heart of thy people, and prepare their heart unto thee:"],
['1Chronicles','1Chr','29','019','19',"And give unto Solomon my son a perfect heart, to keep thy commandments, thy testimonies, and thy statutes, and to do all \emph{these} \emph{things}, and to build the palace, \emph{for} the which I have made provision."],
['1Chronicles','1Chr','29','020','20',"And David said to all the congregation, Now bless the LORD your God. And all the congregation blessed the LORD God of their fathers, and bowed down their heads, and worshipped the LORD, and the king."],
['1Chronicles','1Chr','29','021','21',"And they sacrificed sacrifices unto the LORD, and offered burnt offerings unto the LORD, on the morrow after that day, \emph{even} a thousand bullocks, a thousand rams, \emph{and} a thousand lambs, with their drink offerings, and sacrifices in abundance for all Israel:"],
['1Chronicles','1Chr','29','022','22',"And did eat and drink before the LORD on that day with great gladness. And they made Solomon the son of David king the second time, and anointed \emph{him} unto the LORD \emph{to} \emph{be} the chief governor, and Zadok \emph{to} \emph{be} priest."],
['1Chronicles','1Chr','29','023','23',"Then Solomon sat on the throne of the LORD as king instead of David his father, and prospered; and all Israel obeyed him."],
['1Chronicles','1Chr','29','024','24',"And all the princes, and the mighty men, and all the sons likewise of king David, submitted themselves unto Solomon the king."],
['1Chronicles','1Chr','29','025','25',"And the LORD magnified Solomon exceedingly in the sight of all Israel, and bestowed upon him \emph{such} royal majesty as had not been on any king before him in Israel."],
['1Chronicles','1Chr','29','026','26',"Thus David the son of Jesse reigned over all Israel."],
['1Chronicles','1Chr','29','027','27',"And the time that he reigned over Israel \emph{was} forty years; seven years reigned he in Hebron, and thirty and three \emph{years} reigned he in Jerusalem."],
['1Chronicles','1Chr','29','028','28',"And he died in a good old age, full of days, riches, and honour: and Solomon his son reigned in his stead."],
['1Chronicles','1Chr','29','029','29',"Now the acts of David the king, first and last, behold, they \emph{are} written in the book of Samuel the seer, and in the book of Nathan the prophet, and in the book of Gad the seer,"],
['1Chronicles','1Chr','29','030','30',"With all his reign and his might, and the times that went over him, and over Israel, and over all the kingdoms of the countries."]
]

def main():
    ProcessVerses('FirstChronicles29',FirstChronicles29)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{Furthermore David the king said unto all the congregation, Solomon my son, whom alone God hath chosen, \emph{is} \emph{yet} young and tender, and the work \emph{is} great: for the palace \emph{is} not for man, but for the LORD God.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{Now I have prepared with all my might for the house of my God the gold for \emph{things} \emph{to} \emph{be} \emph{made} of gold, and the silver for \emph{things} of silver, and the brass for \emph{things} of brass, the iron for \emph{things} of iron, and wood for \emph{things} of wood; onyx stones, and \emph{stones} to be set, glistering stones, and of divers colours, and all manner of precious stones, and marble stones in abundance.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{Moreover, because I have set my affection

sojourners & 1 \\ \hline
\emph{were} & 1 \\ \hline
shadow & 1 \\ \hline
\emph{there} & 1 \\ \hline
none & 1 \\ \hline
abiding & 1 \\ \hline
store & 1 \\ \hline
an & 1 \\ \hline
\emph{cometh} & 1 \\ \hline
know & 1 \\ \hline
triest & 1 \\ \hline
hast & 1 \\ \hline
pleasure & 1 \\ \hline
As & 1 \\ \hline
me & 1 \\ \hline
these & 1 \\ \hline
now & 1 \\ \hline
seen & 1 \\ \hline
are & 1 \\ \hline
present & 1 \\ \hline
here & 1 \\ \hline
Abraham & 1 \\ \hline
Isaac & 1 \\ \hline
imagination & 1 \\ \hline
thoughts & 1 \\ \hline
prepare & 1 \\ \hline
commandments & 1 \\ \hline
testimonies & 1 \\ \hline
statutes & 1 \\ \hline
do & 1 \\ \hline
\emph{these} & 1 \\ \hline
\emph{for} & 1 \\ \hline
provision & 1 \\ \hline
bless & 1 \\ \hline
your & 1 \\ \hline
bowed & 1 \\ \hline
down & 1 \\ \hline
heads & 1 \\ \hline
worshipped & 1 \\ \hline
sacrificed & 1 \\ \hline
burnt & 1 \\ \hline
morrow & 1 \\ \hline
\emph{even} & 1 \\ \hline
bullocks & 1 \\ \hline
rams & 1 \\ \hline
\emph{and} & 1 \\ \hline